# Preamble

In [ ]:
import os
import sys, importlib
import pkg_resources
pkg_resources.require("numpy==1.26.1")
pkg_resources.require("pandas==1.4.2")
pkg_resources.require("seaborn==0.12.0")

import logging

from copy import deepcopy
from typing import Tuple

import numpy as np
import pandas as pd

import math
import scipy
from scipy.optimize import curve_fit
from scipy.interpolate import griddata

from statannotations.Annotator import Annotator

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import matplotlib.font_manager
import seaborn as sns

import utils
from utils import \
    circular_mean, circular_mean_length, circular_variance, circular_stddev, \
    apply_p_atic_order

from restore_data import recover_data

In [ ]:
from IPython.display import set_matplotlib_formats
%matplotlib inline

log = logging.getLogger(__name__)

In [ ]:
from IPython.display import set_matplotlib_formats
%matplotlib inline

## Style

In [ ]:
plt.style.context('seaborn-paper')
set_matplotlib_formats('svg')
plt.rcParams.update(plt.rcParamsDefault)

linewidth = 0.25
titlesize = 'medium'
labelsize = 'small'
ticksize = 'x-small'

markersize = 10
scattersize = 5

palette_stages='gist_heat'

plt.rcParams.update({
    'figure.figsize': [5.9/2, 5.9/2 * 2/3],

    'text.usetex': False,
    'font.size': 10,
    'font.family': 'sans-serif',
    'font.sans-serif': 'Helvetica',

    'figure.titlesize': titlesize,
    'legend.title_fontsize': labelsize,
    'legend.fontsize': ticksize,
    'axes.labelsize': labelsize,
    'xtick.labelsize': ticksize,
    'ytick.labelsize': ticksize,
    'ytick.labelsize': ticksize,

    'figure.autolayout': False,

    'axes.linewidth': linewidth,
    'xtick.major.width': linewidth,
    'xtick.minor.width': 0.8*linewidth,
    'ytick.major.width': linewidth,
    'ytick.minor.width': 0.8*linewidth,
    'grid.linewidth': linewidth,

    'patch.linewidth': linewidth,

    'lines.markersize': scattersize
})

In [ ]:
from matplotlib import font_manager
matplotlib.font_manager.findSystemFonts(fontpaths='/usr/share/fonts/truetype',
                                        fontext='ttf')
font_manager.findfont('Helvetica', fallback_to_default=False)

In [ ]:
lineplot_kwargs = dict(
    markers=True,
    ls='',
    dashes=False,
    ci='sd',
    err_style='bars'
)

position_ordering = ["0", "25I", "25S", "50I", "50S", "75I", "75S", "100"]
# stage_ordering = ["E8", "E10", "E12", "E13", "E14"]
stage_ordering = ["E8", "E10", "E12", "E14"]

experiment_order=[
    'MLCKdmso-4hrs',
    'MLCKdmso-16hrs',
    'MLCKml7-4hrs',
    'MLCKml7-16hrs',
    'Vangl2Control',
    'Vangl2KO',
]

treatment_order=[
    'MLCKdmso',
    'MLCKml7',
    'Vangl2Control',
    'Vangl2KO',
    'LiveControl',
    'LiveML7',
]

lineplot_position_stage_kwargs = dict(
    style='position',
    style_order=position_ordering,
    hue='stage',
    hue_order=stage_ordering,
    size='sample_id',
    palette=palette_stages
)
lineplot_position_stage_kwargs.update(lineplot_kwargs)

## Import data

In [ ]:
basepath = os.path.dirname('')
basepath = os.path.abspath(os.path.join(basepath, ".."))
data_basedir = "data"
rel_paths=[
    'HCA-Actn4_new/E8',
    'HCA-Actn4_new/E10',
    'HCA-Actn4_new/E12',
    'HCA-Actn4/E8',
    'HCA-Actn4/E8_2024',
    'HCA-Actn4/E10',
    'HCA-Actn4/E12',
    'HCA-Actn4/E12_2024',
    'HCA-Actn4/E14',
    'HCA-Actn4/E14_2024',
    'Vangl2_KO/Control',
    'Vangl2_KO/Treatment',
    'Vangl2_KO/Treatment_Vangl2',
    'Atoh1/Control',
    'Atoh1/Treatment',
    'MLCK_inhibition/DMSO__4hrs',
    'MLCK_inhibition/DMSO__16hrs',
    'MLCK_inhibition/ML7__4hrs',
    'MLCK_inhibition/ML7__16hrs',
    'ROCK_inhibitions',
    'PTX_inhibitions'
]
data_path = "102_treated_data"
save_file_prefix='summary'

In [ ]:
figure_path = 'analysis/1_figures'
figure_path = os.path.abspath(os.path.join(basepath, figure_path))

In [ ]:
paths = []
for p in rel_paths:
    paths.append(os.path.join(basepath, data_basedir, p, data_path))
cells, bonds = recover_data(paths=paths, save_file_prefix=save_file_prefix)

In [ ]:
# complete p-atic order

for file_id in cells['file_id'].unique():
    __cells = cells.loc[cells['file_id']==file_id].copy()
print(f"Calculating p-atic order of file {file_id} ({__cells['filename'].unique()})")

result = apply_p_atic_order(__cells, ps=np.asarray(range(3, 10)))

for name, values in result.items():
    if 'hexatic_order' in name:
        continue
    cells.loc[(cells["file_id"]==file_id), name] = values

In [ ]:

cells = cells.loc[~cells['is_border_cell']]
if not 'aligned_with_PD_axis' in cells.columns:
    cells['aligned_with_PD_axis'] = False
    bonds['aligned_with_PD_axis'] = False
else:
    cells['aligned_with_PD_axis'] = np.where(
        ~np.isnan(np.asarray(cells['aligned_with_PD_axis'], dtype=float)),
        cells['aligned_with_PD_axis'],
        False
    )
    bonds['aligned_with_PD_axis'] = np.where(
        ~np.isnan(np.asarray(bonds['aligned_with_PD_axis'], dtype=float)),
        bonds['aligned_with_PD_axis'],
        False
    )

if 'Experiment' in cells.columns:
    cells['Experiment'] = cells['Experiment'].where(
        ~cells['Experiment'].isnull(),
        'Control'
    )
    bonds['Experiment'] = bonds['Experiment'].where(
        ~bonds['Experiment'].isnull(),
        'Control'
    )
else:
    cells['Experiment'] = 'Control'
    bonds['Experiment'] = 'Control'

print(cells['Experiment'].unique())

In [ ]:


# Map to treatment
cells['Treatment'] = cells['Experiment']
bonds['Treatment'] = bonds['Experiment']

cells.loc[cells['Experiment']=='Live0Control', 'Treatment'] = 'LiveControl'
cells.loc[cells['Experiment']=='Live150Control', 'Treatment'] = 'LiveControl'
cells.loc[cells['Experiment']=='Live480Control', 'Treatment'] = 'LiveControl'
cells.loc[cells['Experiment']=='Live0ML7', 'Treatment'] = 'LiveML7'
cells.loc[cells['Experiment']=='Live150ML7', 'Treatment'] = 'LiveML7'
cells.loc[cells['Experiment']=='Live400ML7', 'Treatment'] = 'LiveML7'

bonds.loc[bonds['Experiment']=='Live0Control', 'Treatment'] = 'LiveControl'
bonds.loc[bonds['Experiment']=='Live150Control', 'Treatment'] = 'LiveControl'
bonds.loc[bonds['Experiment']=='Live480Control', 'Treatment'] = 'LiveControl'
bonds.loc[bonds['Experiment']=='Live0ML7', 'Treatment'] = 'LiveML7'
bonds.loc[bonds['Experiment']=='Live150ML7', 'Treatment'] = 'LiveML7'
bonds.loc[bonds['Experiment']=='Live400ML7', 'Treatment'] = 'LiveML7'

cells['Experiment_Series'] = cells['Treatment']
cells.loc[cells['Treatment']=='Atoh1control', 'Experiment_Series'] = 'Atoh1'
cells.loc[cells['Treatment']=='Atoh1KO', 'Experiment_Series'] = 'Atoh1'
cells.loc[cells['Treatment']=='MLCKdmso-4hrs', 'Experiment_Series'] = 'MLCK'
cells.loc[cells['Treatment']=='MLCKml7-4hrs', 'Experiment_Series'] = 'MLCK'
cells.loc[cells['Treatment']=='MLCKdmso-16hrs', 'Experiment_Series'] = 'MLCK_16hrs'
cells.loc[cells['Treatment']=='MLCKml7-16hrs', 'Experiment_Series'] = 'MLCK_16hrs'
cells.loc[cells['Treatment']=='ROCKcontrol-4hrs', 'Experiment_Series'] = 'ROCK'
cells.loc[cells['Treatment']=='ROCKy27632-4hrs', 'Experiment_Series'] = 'ROCK'
cells.loc[cells['Treatment']=='ROCKcontrol-16hrs', 'Experiment_Series'] = 'ROCK_16hrs'
cells.loc[cells['Treatment']=='ROCKy27632-16hrs', 'Experiment_Series'] = 'ROCK_16hrs'
cells.loc[cells['Treatment']=='PTXcontrol', 'Experiment_Series'] = 'PTX'
cells.loc[cells['Treatment']=='PTXinhibition', 'Experiment_Series'] = 'PTX'
cells.loc[cells['Treatment']=='Vangl2Control', 'Experiment_Series'] = 'Vangl2'
cells.loc[cells['Treatment']=='Vangl2KO', 'Experiment_Series'] = 'Vangl2'
cells.loc[cells['Treatment']=='LiveControl', 'Experiment_Series'] = 'Live'
cells.loc[cells['Treatment']=='LiveML7', 'Experiment_Series'] = 'Live'

bonds['Experiment_Series'] = bonds['Treatment']
bonds.loc[bonds['Treatment']=='Atoh1control', 'Experiment_Series'] = 'Atoh1'
bonds.loc[bonds['Treatment']=='Atoh1KO', 'Experiment_Series'] = 'Atoh1'
bonds.loc[bonds['Treatment']=='MLCKdmso-4hrs', 'Experiment_Series'] = 'MLCK'
bonds.loc[bonds['Treatment']=='MLCKml7-4hrs', 'Experiment_Series'] = 'MLCK'
bonds.loc[bonds['Treatment']=='MLCKdmso-16hrs', 'Experiment_Series'] = 'MLCK_16hrs'
bonds.loc[bonds['Treatment']=='MLCKml7-16hrs', 'Experiment_Series'] = 'MLCK_16hrs'
bonds.loc[bonds['Treatment']=='ROCKcontrol-4hrs', 'Experiment_Series'] = 'ROCK'
bonds.loc[bonds['Treatment']=='ROCKy27632-4hrs', 'Experiment_Series'] = 'ROCK'
bonds.loc[bonds['Treatment']=='ROCKcontrol-16hrs', 'Experiment_Series'] = 'ROCK_16hrs'
bonds.loc[bonds['Treatment']=='ROCKy27632-16hrs', 'Experiment_Series'] = 'ROCK_16hrs'
bonds.loc[bonds['Treatment']=='PTXcontrol', 'Experiment_Series'] = 'PTX'
bonds.loc[bonds['Treatment']=='PTXinhibition', 'Experiment_Series'] = 'PTX'
bonds.loc[bonds['Treatment']=='Vangl2Control', 'Experiment_Series'] = 'Vangl2'
bonds.loc[bonds['Treatment']=='Vangl2KO', 'Experiment_Series'] = 'Vangl2'
bonds.loc[bonds['Treatment']=='LiveControl', 'Experiment_Series'] = 'Live'
bonds.loc[bonds['Treatment']=='LiveML7', 'Experiment_Series'] = 'Live'

{figure_path}
# Assign is_control
cells['is_Control'] = False
cells.loc[cells['Treatment']=='Control', 'is_Control'] = True
cells.loc[cells['Treatment']=='Atoh1control', 'is_Control'] = True
cells.loc[cells['Treatment']=='Atoh1KO', 'is_Control'] = False
cells.loc[cells['Treatment']=='MLCKdmso-4hrs', 'is_Control'] = True
cells.loc[cells['Treatment']=='MLCKml7-4hrs', 'is_Control'] = False
cells.loc[cells['Treatment']=='MLCKdmso-16hrs', 'is_Control'] = True
cells.loc[cells['Treatment']=='MLCKml7-16hrs', 'is_Control'] = False
cells.loc[cells['Treatment']=='ROCKcontrol-4hrs', 'is_Control'] = True
cells.loc[cells['Treatment']=='ROCKy27632-4hrs', 'is_Control'] = False
cells.loc[cells['Treatment']=='ROCKcontrol-16hrs', 'is_Control'] = True
cells.loc[cells['Treatment']=='ROCKy27632-16hrs', 'is_Control'] = False
cells.loc[cells['Treatment']=='PTXcontrol', 'is_Control'] = True
cells.loc[cells['Treatment']=='PTXinhibition', 'is_Control'] = False
cells.loc[cells['Treatment']=='Vangl2Control', 'is_Control'] = True
cells.loc[cells['Treatment']=='Vangl2KO', 'is_Control'] = False
cells.loc[cells['Treatment']=='LiveControl', 'is_Control'] = True
cells.loc[cells['Treatment']=='LiveML7', 'is_Control'] = False

bonds['is_Control'] = False
bonds.loc[bonds['Treatment']=='Control', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='Atoh1control', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='Atoh1KO', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='MLCKdmso-4hrs', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='MLCKml7-4hrs', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='MLCKdmso-16hrs', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='MLCKml7-16hrs', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='ROCKcontrol-4hrs', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='ROCKy27632-4hrs', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='ROCKcontrol-16hrs', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='ROCKy27632-16hrs', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='PTXcontrol', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='PTXinhibition', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='Vangl2Control', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='Vangl2KO', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='LiveControl', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='LiveML7', 'is_Control'] = False

### Correct orientations

In [ ]:
# correct orientations
print("WARNING reorienting cilia orientations, but not other orientations")
mask_rotate90 = (
    (cells['filename'] == 'MLCKdmso-4hrs_2.tif')
)
cells.loc[mask_rotate90, 'cilium_phi'] -= math.pi/2
cells.loc[mask_rotate90, 'cilium_phi_corrected'] -= math.pi/2

mask_flip_SI = (
    (cells['Experiment'] == 'MLCKdmso-4hrs') |
    (
        (cells['Experiment'] == 'Control') &
        (cells['stage'] == 'E14') &
        (cells['position'] == '75I')
    )
)
print(mask_flip_SI.sum())
cells.loc[mask_flip_SI, 'cilium_phi'] *= -1
cells.loc[mask_flip_SI, 'cilium_phi_corrected'] *= -1


# correct y-axis pointing down
cells['cilium_phi'] *= -1
cells['cilium_phi_corrected'] *= -1


cells['cilium_DX'] = cells['cilium_rho'] * np.cos(cells['cilium_phi'])
cells['cilium_DY'] = cells['cilium_rho'] * np.sin(cells['cilium_phi'])
cells['cilium_DX_corrected'] = cells['cilium_rho_corrected'] * np.cos(cells['cilium_phi_corrected'])
cells['cilium_DY_corrected'] = cells['cilium_rho_corrected'] * np.sin(cells['cilium_phi_corrected'])

### Corrected neighbourhood

In [ ]:
# Cells by type in Ex-ovo data
HC = cells.loc[(cells['is_HC'])  & (cells['Experiment'] == 'Control')]
SC = cells.loc[(~cells['is_HC']) & (cells['Experiment'] == 'Control')]

# Linear fit neighbour number to cell area
m_HC = np.polyfit(HC['normalized_area_cells'], HC['num_neighbors'], 1)[0]
m_SC = np.polyfit(SC['normalized_area_cells'], SC['num_neighbors'], 1)[0]

cells['num_neighbors__corr'] = np.where(
    cells['is_HC'],
    cells['num_neighbors'] + m_HC * (1 - cells['normalized_area_cells']),
    cells['num_neighbors'] + m_SC * (1 - cells['normalized_area_cells']),
)

HC = None
SC = None
del HC, SC

### Align tissue axes

In [ ]:
# Average orientations per image
grp=[
    'Experiment_Series',
    'stage',
    'position',
    'is_Control',
    'filename',
    'file_id'
]
grp_data = cells.copy().groupby(by=grp)

p_vec = ['cilium_DX_corrected', 'cilium_DY_corrected']

results = pd.DataFrame()
results['mean_orientation'] = grp_data[p_vec].apply(circular_mean, eulerian=True)
results['circular_stddev'] = grp_data[p_vec].apply(circular_stddev, eulerian=True)
results['circular_variance'] = grp_data[p_vec].apply(circular_variance, eulerian=True)
results['resulting_length'] = grp_data[p_vec].apply(circular_mean_length, eulerian=True)
results['datapoints'] = grp_data['cilium_DX_corrected'].apply(lambda d: d.count())
results['aligned_with_PD_axis'] = grp_data['aligned_with_PD_axis'].all()

results['mean_orientation__angles'] = (results['mean_orientation'] / math.pi * 180 + 360) % 360
results['circular_stddev__angles'] = (results['circular_stddev'] / math.pi * 180 + 360) % 360

results.to_csv("{}/{}/{}_polarity.csv".format(basepath, data_basedir, save_file_prefix))
results = results.reset_index()

grp_data = None
del grp_data

In [ ]:
# align cilium_phi by circular mean of mean orientation
cells['cilium_phi_corrected__aligned'] = cells['cilium_phi_corrected'].copy()
for file_id in cells['file_id'].unique():
    cells.loc[cells['file_id']==file_id, 'cilium_phi_corrected__aligned'] -= results.loc[results['file_id']==file_id, 'mean_orientation'].iloc[0]

for series in results['Experiment_Series'].unique():
    for is_Control in results['is_Control'].unique():
        mask = (
            (results['Experiment_Series']==series) &
            (results['is_Control']==is_Control) &
            (results['aligned_with_PD_axis'])
        )

        cmean = circular_mean(
            results.loc[mask, ['resulting_length', 'mean_orientation']],
            eulerian=False
        )

        mask = (
            (cells['Experiment_Series']==series) &
            (cells['is_Control']==is_Control)
        )
        cells.loc[mask, 'cilium_phi_corrected__aligned'] += cmean

cells['cilium_DX_corrected__aligned'] = cells['cilium_rho_corrected'] * np.cos(cells['cilium_phi_corrected__aligned'])
cells['cilium_DY_corrected__aligned'] = cells['cilium_rho_corrected'] * np.sin(cells['cilium_phi_corrected__aligned'])


# center aligned cilium_phi to inferior (90 deg) orientation
cells['cilium_phi_corrected__centred'] = cells['cilium_phi_corrected'].copy()
for file_id in cells['file_id'].unique():
    cells.loc[cells['file_id']==file_id, 'cilium_phi_corrected__centred'] -= results.loc[results['file_id']==file_id, 'mean_orientation'].iloc[0] + math.pi/2.

cells['cilium_DX_corrected__centred'] = cells['cilium_rho_corrected'] * np.cos(cells['cilium_phi_corrected__centred'])
cells['cilium_DY_corrected__centred'] = cells['cilium_rho_corrected'] * np.sin(cells['cilium_phi_corrected__centred'])


In [ ]:
# Average orientations per image type
grp=[
    'Experiment_Series',
    'stage',
    'position',
    'is_Control',
]
grp_data = cells.copy().groupby(by=grp)

p_vec = ['cilium_DX_corrected', 'cilium_DY_corrected']


results_global_mean = pd.DataFrame()
grp_global_mean=grp[:-2]
results_global_mean['mean_orientation'] = grp_data[p_vec].apply(circular_mean, eulerian=True)
results_global_mean['circular_stddev'] = grp_data[p_vec].apply(circular_stddev, eulerian=True)
results_global_mean['circular_variance'] = grp_data[p_vec].apply(circular_variance, eulerian=True)
results_global_mean['resulting_length'] = grp_data[p_vec].apply(circular_mean_length, eulerian=True)
results_global_mean['datapoints'] = grp_data['cilium_DX_corrected'].apply(lambda d: d.count())

results_global_mean['mean_orientation__angles'] = (results_global_mean['mean_orientation'] / math.pi * 180 + 360) % 360
results_global_mean['circular_stddev__angles'] = (results_global_mean['circular_stddev'] / math.pi * 180 + 360) % 360

results_global_mean.to_csv("{}/{}/{}_polarity__global_mean.csv".format(basepath, data_basedir, save_file_prefix))

grp_data = None
results_global_mean = None
del grp_data, results_global_mean

In [ ]:
# Average orientations in aligned images

grp_data = cells.copy().groupby(by=grp)

p_vec = ['cilium_DX_corrected__aligned', 'cilium_DY_corrected__aligned']

results_aligned = pd.DataFrame()
results_aligned['mean_orientation'] = grp_data[p_vec].apply(circular_mean, eulerian=True)
results_aligned['circular_stddev'] = grp_data[p_vec].apply(circular_stddev, eulerian=True)
results_aligned['circular_variance'] = grp_data[p_vec].apply(circular_variance, eulerian=True)
results_aligned['resulting_length'] = grp_data[p_vec].apply(circular_mean_length, eulerian=True)
results_aligned['datapoints'] = grp_data['cilium_DX_corrected__aligned'].apply(lambda d: d.count())

results_aligned['mean_orientation__angles'] = (results_aligned['mean_orientation'] / math.pi * 180 + 360) % 360
results_aligned['circular_stddev__angles'] = (results_aligned['circular_stddev'] / math.pi * 180 + 360) % 360

results_aligned.to_csv("{}/{}/{}_polarity__aligned.csv".format(basepath, data_basedir, save_file_prefix))

grp_data = None
results_aligned = None
del grp_data, results_aligned

In [ ]:
results = None
del results

### Tissue avearges

In [ ]:
def tissue_averages(cells, bonds, tissue_grp):
    """Calculates numerous average quantities based on the tissue_grp groups
    """
    cells = cells.loc[~cells['is_border_cell']]
    HC = cells.loc[cells['is_HC']].copy()
    SC = cells.loc[~cells['is_HC']].copy()

    tissue_data = pd.DataFrame()

    # count cells
    tissue_data['num_HC'] = HC.groupby(by=tissue_grp)['is_HC'].count()
    tissue_data['num_SC'] = SC.groupby(by=tissue_grp)['is_HC'].count()
    tissue_data['num_cells'] = cells.groupby(by=tissue_grp)['is_HC'].count()

    tissue_data['HC_density'] = tissue_data['num_HC'] / tissue_data['num_cells']
    tissue_data['number_ratio'] = tissue_data['num_HC'] / tissue_data['num_SC']

    # cell areas
    tissue_data['HC_area'] = HC.groupby(by=tissue_grp)['normalized_area_cells'].mean()
    tissue_data['HC_area__std'] = HC.groupby(by=tissue_grp)['normalized_area_cells'].std()

    tissue_data['SC_area'] = SC.groupby(by=tissue_grp)['normalized_area_cells'].mean()
    tissue_data['SC_area__std'] = SC.groupby(by=tissue_grp)['normalized_area_cells'].std()

    tissue_data['area__no_norm'] = cells.groupby(by=tissue_grp)['area'].mean()
    tissue_data['area__no_norm__std'] = cells.groupby(by=tissue_grp)['area'].std()

    tissue_data['HC_area__no_norm'] = HC.groupby(by=tissue_grp)['area'].mean()
    tissue_data['HC_area__no_norm__std'] = HC.groupby(by=tissue_grp)['area'].std()

    tissue_data['SC_area__no_norm'] = SC.groupby(by=tissue_grp)['area'].mean()
    tissue_data['SC_area__no_norm__std'] = SC.groupby(by=tissue_grp)['area'].std()


    tissue_data['relative_area'] = (
        HC.groupby(by=tissue_grp)['normalized_area_cells'].mean()
        / SC.groupby(by=tissue_grp)['normalized_area_cells'].mean()
    )
    tissue_data['relative_area__std'] = tissue_data['relative_area'] * abs(
        (  HC.groupby(by=tissue_grp)['normalized_area_cells'].std()
        / HC.groupby(by=tissue_grp)['normalized_area_cells'].mean())
        -
        (  SC.groupby(by=tissue_grp)['normalized_area_cells'].std()
        / SC.groupby(by=tissue_grp)['normalized_area_cells'].mean())
    )
    tissue_data['HC_area_coverage'] = tissue_data['HC_area'] * tissue_data['HC_density']


    # neighborhood
    tissue_data['neighbor_number'] = cells.groupby(by=tissue_grp)['num_neighbors'].mean()
    tissue_data['neighbor_number__std'] = cells.groupby(by=tissue_grp)['num_neighbors'].std()
    tissue_data['neighbor_number__corr'] = cells.groupby(by=tissue_grp)['num_neighbors__corr'].mean()
    tissue_data['neighbor_number__corr__std'] = cells.groupby(by=tissue_grp)['num_neighbors__corr'].std()

    tissue_data['coordination_number'] = HC.groupby(by=tissue_grp)['num_neighbors'].mean()
    tissue_data['coordination_number__std'] = HC.groupby(by=tissue_grp)['num_neighbors'].std()
    tissue_data['coordination_number__corr'] = HC.groupby(by=tissue_grp)['num_neighbors__corr'].mean()
    tissue_data['coordination_number__corr__std'] = HC.groupby(by=tissue_grp)['num_neighbors__corr'].std()

    tissue_data['subordination_number'] = SC.groupby(by=tissue_grp)['num_neighbors'].mean()
    tissue_data['subordination_number__std'] = SC.groupby(by=tissue_grp)['num_neighbors'].std()
    tissue_data['subordination_number__corr'] = SC.groupby(by=tissue_grp)['num_neighbors__corr'].mean()
    tissue_data['subordination_number__corr__std'] = SC.groupby(by=tissue_grp)['num_neighbors__corr'].std()

    tissue_data['HH_contacts'] = HC.groupby(by=tissue_grp)['num_hair_neighbors'].mean()
    tissue_data['HH_contacts__std'] = HC.groupby(by=tissue_grp)['num_hair_neighbors'].std()
    tissue_data['HH_contacts_total'] = HC.groupby(by=tissue_grp)['num_hair_neighbors'].sum()

    tissue_data['SH_contacts'] = SC.groupby(by=tissue_grp)['num_hair_neighbors'].mean()
    tissue_data['SH_contacts__std'] = SC.groupby(by=tissue_grp)['num_hair_neighbors'].std()


    tissue_data['num_HH_bonds'] = np.asarray(bonds.groupby(by=tissue_grp).apply(lambda bonds: (bonds['type'] == 'HH').sum()), dtype=int)
    tissue_data['num_HS_bonds'] = np.asarray(bonds.groupby(by=tissue_grp).apply(lambda bonds: (bonds['type'] == 'HS').sum()), dtype=int)
    tissue_data['num_SS_bonds'] = np.asarray(bonds.groupby(by=tissue_grp).apply(lambda bonds: (bonds['type'] == 'SS').sum()), dtype=int)
    tissue_data['num_HC_bonds'] = tissue_data['num_HH_bonds'] + tissue_data['num_HS_bonds']
    tissue_data['num_bonds'] = tissue_data['num_HH_bonds'] + tissue_data['num_HS_bonds'] + tissue_data['num_SS_bonds']
    tissue_data['fraction_HH_bonds'] = tissue_data['num_HH_bonds'] / tissue_data['num_bonds']

    norm = (cells.groupby(by=tissue_grp)['area_cells'].mean() / math.pi)**0.5
    tissue_data['bonds_normalised_length'] = bonds.groupby(by=tissue_grp)['bond_length_in_px'].mean() / norm
    tissue_data['bonds_normalised_length__std'] = bonds.groupby(by=tissue_grp)['bond_length_in_px'].std() / norm
    tissue_data['HS_bonds_normalised_length'] = bonds.loc[bonds['type']=='HS'].groupby(by=tissue_grp)['bond_length_in_px'].mean() / norm
    tissue_data['HS_bonds_normalised_length__std'] = bonds.loc[bonds['type']=='HS'].groupby(by=tissue_grp)['bond_length_in_px'].std() / norm
    tissue_data['SS_bonds_normalised_length'] = bonds.loc[bonds['type']=='SS'].groupby(by=tissue_grp)['bond_length_in_px'].mean() / norm
    tissue_data['SS_bonds_normalised_length__std'] = bonds.loc[bonds['type']=='SS'].groupby(by=tissue_grp)['bond_length_in_px'].std() / norm


    # spatial organisation
    tissue_data['hexatic_order'] = HC.groupby(by=tissue_grp)['hexatic_order'].mean()
    tissue_data['hexatic_order__std'] = HC.groupby(by=tissue_grp)['hexatic_order'].std()
    tissue_data['hexatic_order_corrected'] = HC.groupby(by=tissue_grp)['hexatic_order_corrected'].mean()
    tissue_data['hexatic_order_corrected__std'] = HC.groupby(by=tissue_grp)['hexatic_order_corrected'].std()
    tissue_data['hexatic_order_ellipse_fit'] = HC.groupby(by=tissue_grp)['hexatic_order_ellipse_fit'].mean()
    tissue_data['hexatic_order_ellipse_fit__std'] = HC.groupby(by=tissue_grp)['hexatic_order_ellipse_fit'].std()
    for p in range(3, 10):
        tissue_data[f'{p}-atic_order'] = HC.groupby(by=tissue_grp)[f'{p}-atic_order'].mean()
        tissue_data[f'{p}-atic_order__std'] = HC.groupby(by=tissue_grp)[f'{p}-atic_order'].std()
        tissue_data[f'{p}-atic_order_corrected'] = HC.groupby(by=tissue_grp)[f'{p}-atic_order_corrected'].mean()
        tissue_data[f'{p}-atic_order_corrected__std'] = HC.groupby(by=tissue_grp)[f'{p}-atic_order_corrected'].std()
        tissue_data[f'{p}-atic_order_ellipse_fit'] = HC.groupby(by=tissue_grp)[f'{p}-atic_order_ellipse_fit'].mean()
        tissue_data[f'{p}-atic_order_ellipse_fit__std'] = HC.groupby(by=tissue_grp)[f'{p}-atic_order_ellipse_fit'].std()


    tissue_data['num_next_HC_neighbors'] = HC.groupby(by=tissue_grp)['num_next_HC_neighbors'].mean()
    tissue_data['num_next_HC_neighbors__std'] = HC.groupby(by=tissue_grp)['num_next_HC_neighbors'].std()


    tissue_data['elongation'] = cells.groupby(by=tissue_grp)['elongation_cells'].mean()
    tissue_data['elongation_HC'] = HC.groupby(by=tissue_grp)['elongation_cells'].mean()
    tissue_data['elongation_SC'] = SC.groupby(by=tissue_grp)['elongation_cells'].mean()
    


    # polarity
    tissue_data['num_cilia'] = cells.groupby(by=tissue_grp)['cilium_DX'].count()

    tissue_data['polarity'] = HC.groupby(by=tissue_grp)[['cilium_DX', 'cilium_DY']].apply(circular_mean)
    tissue_data['polarity_angles'] = tissue_data['polarity'] / math.pi * 180.
    tissue_data['polarity__variance'] = HC.groupby(by=tissue_grp)[['cilium_DX', 'cilium_DY']].apply(circular_variance)

    HC['cilium_DX_corrected'] = HC['cilium_rho_corrected'] * np.cos(HC['cilium_phi_corrected'])
    HC['cilium_DY_corrected'] = HC['cilium_rho_corrected'] * np.sin(HC['cilium_phi_corrected'])
    tissue_data['polarity_corrected'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean)
    tissue_data['polarity_corrected_angles'] = tissue_data['polarity_corrected'] / math.pi * 180.
    tissue_data['polarity_corrected__stddev'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_stddev)
    tissue_data['polarity_corrected__stddev_angles'] = tissue_data['polarity_corrected__stddev'] / math.pi * 180
    tissue_data['polarity_corrected__variance'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_variance)

    if 'cilium_DX_corrected__aligned' in tissue_data.columns:
        tissue_data['polarity_corrected_aligned'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected__aligned', 'cilium_DY_corrected__aligned']].apply(circular_mean)
        tissue_data['polarity_corrected_aligned_angles'] = tissue_data['polarity_corrected_aligned'] / 180 * math.pi
        tissue_data['polarity_corrected_aligned__stddev'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected__aligned', 'cilium_DY_corrected__aligned']].apply(circular_stddev)
        tissue_data['polarity_corrected_aligned__stddev_angles'] = np.where(
            tissue_data['num_cilia'] >= 10,
            tissue_data['polarity_corrected_aligned__stddev'] / math.pi * 180,
            np.nan
        )
        tissue_data['polarity_corrected_aligned__variance'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected__aligned', 'cilium_DY_corrected__aligned']].apply(circular_variance)

    mask = tissue_data['num_cilia'] < 10
    tissue_data.loc[mask, 'polarity'] = np.nan
    tissue_data.loc[mask, 'polarity__variance'] = np.nan
    tissue_data.loc[mask, 'polarity_corrected'] = np.nan
    tissue_data.loc[mask, 'polarity_corrected__variance'] = np.nan
    tissue_data.loc[mask, 'polarity_corrected__stddev'] = np.nan


    return tissue_data.reset_index()

In [ ]:
__tissue_grp = [
    'Experiment',
    'Experiment_Series',
    'Treatment',
    'is_Control',
    'aligned_with_PD_axis',
    'stage',
    'Stage',
    'position',
    'SI_position',
    'S-I position',
    'SI_position_long',
    'PD_position',
    'P-D position',
]

tissue_grp = [
    'file_id',
    'sample_id',
    'filename',
    'filepath',
]

for g in __tissue_grp:
    tissue_grp.append(g)

In [ ]:
tissue_data = tissue_averages(cells, bonds, tissue_grp)

### Summary and export

In [ ]:
cells_experiments = cells.copy()
bonds_experiments = bonds.copy()

cells = cells.loc[cells['Experiment'] == 'Control']
bonds = bonds.loc[bonds['Experiment'] == 'Control']

# select subsets
HC = cells.loc[cells['is_HC']].copy()
SC = cells.loc[~cells['is_HC']].copy()

tissue_data__experiments = tissue_data.copy()
tissue_data = tissue_data.loc[tissue_data['Experiment'] == 'Control']

In [ ]:
out_path = "{}/{}/{}".format(basepath, data_basedir, save_file_prefix)
cells.to_csv("{}_cells.csv".format(out_path))
bonds.to_csv("{}_bonds.csv".format(out_path))

cells_experiments.to_csv("{}_cells_w_treatments.csv".format(out_path))
bonds_experiments.to_csv("{}_bonds_w_treatments.csv".format(out_path))

tissue_data.to_csv("{}_tissue_averages.csv".format(out_path))
tissue_data__experiments.to_csv("{}_tissue_averages_w_treatments.csv".format(out_path))

In [ ]:
print("Number of data-points")

grp = ['stage', 'position']

data_information = pd.DataFrame()
data_information['cells'] = cells.groupby(by=grp)[['num_neighbors']].count()
data_information['HC (cilia)'] = HC.groupby(by=['stage', 'position']).apply(
    lambda hc: (hc['num_neighbors'].count(),
                hc['cilium_DX'].count())
)
data_information['SC'] = SC.groupby(by=grp)[['num_neighbors']].count()
data_information['tissues (w. cilia)'] = HC.groupby(by=grp).apply(
    lambda hc: (len(np.unique(hc['file_id'])),
                len(np.unique(hc.loc[~np.isnan(hc['cilium_DX']), 'file_id'])))
)

print(data_information.to_string())
data_information.to_csv("{}_cells_infos.csv".format(out_path))

data_information = None
del data_information

In [ ]:
print("\nNumber of data-points with known P-D axis")
aligned_cells = cells.loc[cells['aligned_with_PD_axis']].copy()

data_information = pd.DataFrame()
data_information['cells'] = aligned_cells.groupby(by=grp)[['num_neighbors']].count()
data_information['HC (cilia)'] = aligned_cells.loc[aligned_cells['is_HC']].groupby(by=grp).apply(
    lambda hc: (hc['num_neighbors'].count(), hc['cilium_DX'].count()))
data_information['SC'] = aligned_cells.loc[~aligned_cells['is_HC']].groupby(by=grp)[['num_neighbors']].count()
data_information['tissues (w. cilia)'] = aligned_cells.loc[aligned_cells['is_HC']].groupby(by=grp).apply(
    lambda hc: (len(np.unique(hc['file_id'])), len(np.unique(hc.loc[~np.isnan(hc['cilium_DX']), 'file_id']))))

print(data_information.to_string())
data_information.to_csv("{}_cells_aligned_infos.csv".format(out_path))

aligned_cells = None
data_information = None
del aligned_cells, data_information

In [ ]:
print("\nNumber of data-points treatments")

grp = ['Experiment', 'stage', 'position']

data_information = pd.DataFrame()

data_information['cells'] = cells_experiments.groupby(by=grp)[['num_neighbors']].count()
data_information['HC (cilia)'] = cells_experiments.loc[cells_experiments['is_HC']].groupby(by=grp).apply(
    lambda hc: (hc['num_neighbors'].count(), hc['cilium_DX'].count()))
data_information['SC'] = cells_experiments.loc[~cells_experiments['is_HC']].groupby(by=grp)[['num_neighbors']].count()
data_information['tissues (w. cilia)'] = cells_experiments.loc[cells_experiments['is_HC']].groupby(by=grp).apply(
    lambda hc: (len(np.unique(hc['file_id'])), len(np.unique(hc.loc[~np.isnan(hc['cilium_DX']), 'file_id']))))

print(data_information.to_string())
data_information.to_csv("{}_cells_w_treatments_infos.csv".format(out_path))

data_information = None
del data_information

In [ ]:
data = pd.DataFrame()
data['Series'] = tissue_data__experiments['Experiment_Series']
data['Experiment'] = tissue_data__experiments['Experiment']
data['Experiment'] = tissue_data__experiments['Experiment']
data['Name of the experiment'] = tissue_data__experiments.apply(lambda data: f"{data['stage']} {data['position']}" if data['Experiment_Series']=='Control' else data['Experiment'], axis=1)
data['Number of HCs (cilia) / SCs examined'] = tissue_data__experiments.apply(lambda data: f"{data['num_HC']} ({data['num_cilia']}) / {data['num_SC']}", axis=1)
data['Surface area of HCs'] = tissue_data__experiments['HC_area__no_norm']
data['Surface area of SCs'] = tissue_data__experiments['SC_area__no_norm']
data['Average HC neighbour number'] = tissue_data__experiments['coordination_number']
data['Corrected HC neighbour number'] = tissue_data__experiments['coordination_number__corr']
data['Average hexatic order (corrected)'] = tissue_data__experiments['hexatic_order_corrected']
data['Mean orientation'] = tissue_data__experiments['polarity_corrected_angles']
data['CSD'] = tissue_data__experiments['polarity_corrected__stddev_angles']

def rename_experiment(name):
    if name == 'Vangl2Control':
        return 'Vangl2 Control'
    if name == 'Vangl2KO':
        return 'Vangl2 KO'
    if name == 'Atoh1control':
        return 'Atoh1 Control'
    if name == 'Atoh1KO':
        return 'Atoh1 KO'
    if name == 'MLCKdmso-4hrs':
        return 'DMSO 4 hrs'
    if name == 'MLCKdmso-16hrs':
        return 'DMSO 16 hrs'
    if name == 'MLCKml7-4hrs':
        return 'ML7 4 hrs'
    if name == 'MLCKml7-16hrs':
        return 'ML7 4 + 12 hrs'
    if name == 'ROCKcontrol-4hrs':
        return 'DMEM 4 hrs'
    if name == 'ROCKcontrol-16hrs':
        return 'DMEM 16 hrs'
    if name == 'ROCKy27632-4hrs':
        return 'Y-27632 4 hrs'
    if name == 'ROCKy27632-16hrs':
        return 'Y-27632 4 + 12 hrs'
    if name == 'PTXcontrol':
        return 'DMEM 4 hrs (PTX control)'
    if name == 'PTXinhibition':
        return 'PTX 4 hrs'
    else:
        return name
data['Name of the experiment'] = data['Name of the experiment'].apply(rename_experiment)

data.loc[data['Series']=='Control'].to_csv(f"{basepath}/{data_basedir}/{save_file_prefix}_tissue_averages_publication_BP.csv")
data.loc[(data['Series']=='Control') & ((tissue_data__experiments['position']=='25I') | (tissue_data__experiments['position']=='75I'))].to_csv(f"{basepath}/{data_basedir}/{save_file_prefix}_tissue_averages_publication_BP_25I_75I.csv")
data.loc[data['Series']!='Control'].to_csv(f"{basepath}/{data_basedir}/{save_file_prefix}_tissue_averages_publication_treatments.csv")

In [ ]:
print("WARNING tissues with more than 5% H-H contacts (false contacts)")
print(tissue_data__experiments.sort_values(by='fraction_HH_bonds', ascending=False)[['filename', 'file_id', 'is_Control', 'aligned_with_PD_axis', 'fraction_HH_bonds', 'num_HH_bonds']].to_string())

# Schematics

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[3, 3])

phi = np.linspace(0, 2 * math.pi, 101)

long = 3
short = 1
rotate = -30/180*math.pi - math.pi
angle = 15

def __rescale_dx_dy(displ_x, displ_y, *, long, short, theta, scatter=False):
    """Rescale to circle for correction"""
    dx_ = (displ_x * np.cos(-theta) - displ_y * np.sin(-theta)) / (long / short)
    dy_ =  displ_x * np.sin(-theta) + displ_y * np.cos(-theta)

    dx = dx_ * np.cos(theta) - dy_ * np.sin(theta)
    dy = dx_ * np.sin(theta) + dy_ * np.cos(theta)

    rho = (long * short)**0.5

    return dx/short, dy/short

def Rho_ellipse(angle, *, short, long, rotate):
    return long * short / (long**2 * np.sin(angle - rotate)**2 + short**2 * np.cos(angle - rotate)**2)**0.5

# the circle with equal area
area = math.pi * long * short
rho = (area / math.pi)**0.5
ax.plot(rho * np.cos(phi), rho * np.sin(phi), 
        color='black', linestyle='dashed', lw=0.5)

rho_ellipse = Rho_ellipse(phi, short=short, long=long, rotate=rotate)
ax.plot(rho_ellipse * np.cos(phi), rho_ellipse * np.sin(phi),
        color='black', lw=0.5)

cilium_angle = (np.array([10]))/180 * math.pi + rotate
cilium_rho = 0.85 * Rho_ellipse(cilium_angle, 
                                short=short, long=long, rotate=rotate)
ax.scatter(
    cilium_rho* np.cos(cilium_angle), cilium_rho * np.sin(cilium_angle), 
    color='red', s=20
)

dx, dy = __rescale_dx_dy(
    cilium_rho * np.cos(cilium_angle), cilium_rho * np.sin(cilium_angle), 
    long=long, short=short, theta=rotate, scatter=True)
ax.quiver(
    np.zeros_like(dx), np.zeros_like(dx), dx*rho, dy*rho, 
    scale=1, angles='xy', scale_units='xy', color='red'
)

__rho = Rho_ellipse(0, short=short, long=long, rotate=0)
x, y = __rho * np.cos(rotate), __rho * np.sin(rotate)
ax.quiver(
    x, y, -0.9 * x / __rho * (long - rho), -0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', pivot='tip', color='gray'
)
ax.quiver(
    -x, -y, 0.9 * x / __rho * (long - rho), 0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', pivot='tip', color='gray'
)

__rho = Rho_ellipse(90, short=short, long=long, rotate=0)
x, y = -__rho * np.sin(rotate), __rho * np.cos(rotate)
ax.quiver(
    x, y, 0.9 * x / __rho * (long - rho), 0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', color='gray'
)
ax.quiver(
    -x, -y, -0.9 * x / __rho * (long - rho), -0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', color='gray'
)

ax.set_xlim([-4, 4])
ax.set_ylim([-3, 3])
ax.set_aspect('equal')


ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax.quiver(
    0, 0, 2.5*long, 0, 
    scale=1, angles='xy', scale_units='xy', color='black', 
    width=0.0025, pivot='mid'
)
ax.quiver(
    0, 0, 0, long*1.75, 
    scale=1, angles='xy', scale_units='xy', color='black', 
    width=0.0025, pivot='mid'
)

plt.savefig(f"{figure_path}/cell_elongation_correction.svg",format='svg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[3, 3])

# the circle with equal area
area = math.pi * long * short
rho = (area / math.pi)**0.5
ax.plot(rho * np.cos(phi), rho * np.sin(phi), 
        color='black', linestyle='dashed', lw=0.5)

rho_ellipse = Rho_ellipse(phi, short=short, long=long, rotate=rotate)
ax.plot(rho_ellipse * np.cos(phi), rho_ellipse * np.sin(phi), 
        color='black', lw=0.5)

# cilium_angle = (np.linspace(0, 90, 4))/180 * math.pi + rotate
cilium_angle = (np.linspace(0, 360, 6, endpoint=False)+15)/180*math.pi + rotate
cilium_rho = Rho_ellipse(cilium_angle, short=short, long=long, rotate=rotate)
_dx, _dy = __rescale_dx_dy(
    cilium_rho * np.cos(cilium_angle), cilium_rho * np.sin(cilium_angle), 
    long=short, short=long, theta=rotate, scatter=True
)
cilium_angle = np.arctan2(_dy, _dx)[2:5]
cilium_rho = 2. * Rho_ellipse(cilium_angle, 
                              short=short, long=long, rotate=rotate)

dx, dy = __rescale_dx_dy(
    cilium_rho * np.cos(cilium_angle), cilium_rho * np.sin(cilium_angle),
    long=long, short=short, theta=rotate, scatter=True
)
print(np.arctan2(dy, dx)/math.pi * 180)
ax.scatter(
    cilium_rho/2* np.cos(cilium_angle), cilium_rho/2 * np.sin(cilium_angle), 
    color='blue', s=20
)
ax.quiver(
    np.zeros_like(dx), np.zeros_like(dx), 
    cilium_rho* np.cos(cilium_angle), cilium_rho * np.sin(cilium_angle), 
    scale=1, angles='xy', scale_units='xy', color='blue'
)
ax.scatter(dx*rho/2, dy*rho/2, color='red')
ax.quiver(
    np.zeros_like(dx), np.zeros_like(dx), dx*rho, dy*rho, 
    scale=1, angles='xy', scale_units='xy', color='red'
)

__rho = Rho_ellipse(0, short=short, long=long, rotate=0)
x, y = __rho * np.cos(rotate), __rho * np.sin(rotate)
ax.quiver(
    x, y, -0.9 * x / __rho * (long - rho), -0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', pivot='tip', color='gray'
)
ax.quiver(
    -x, -y, 0.9 * x / __rho * (long - rho), 0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', pivot='tip', color='gray'
)

__rho = Rho_ellipse(90, short=short, long=long, rotate=0)
x, y = -__rho * np.sin(rotate), __rho * np.cos(rotate)
ax.quiver(
    x, y, 0.9 * x / __rho * (long - rho), 0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', color='gray'
)
ax.quiver(
    -x, -y, -0.9 * x / __rho * (long - rho), -0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', color='gray'
)

ax.set_xlim([-4, 4])
ax.set_ylim([-3, 3])
ax.set_aspect('equal')


ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
# ax.axis('off')

ax.quiver(
    0, 0, 2.5*long, 0, 
    scale=1, angles='xy', scale_units='xy', color='black', 
    width=0.0025, pivot='mid'
)
ax.quiver(
    0, 0, 0, long*1.75, 
    scale=1, angles='xy', scale_units='xy', color='black', 
    width=0.0025, pivot='mid'
)

plt.savefig(f"{figure_path}/cell_hexatic_correction.svg",format='svg')

# Analysis

## Area

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[5.9/2, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['position'] == '25I'],
    ax=ax1,
    x='stage',
    y='area',
    hue='is_HC',
    hue_order=[True, False],
    palette=['gray', 'red'],
    join=False,
    errorbar='sd',
    # size='stage',
    # sizes=[2],
    # capsize=.05,
    dodge=True,
    errwidth=linewidth
)
plt.setp(points.collections, sizes=[markersize])


ax1.set_ylim([0, None])
ax1.set_xlim([-0.5, 3.5])

ax1.set_ylabel('Cell area [$\mathrm{\mu m^2}$]')
ax1.set_ylim(0, 70)
ax1.set_yticks([0, 20, 40, 60])

ax1.set_xlabel('')
ax1.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

leg_handles = ax1.get_legend_handles_labels()[0]
ax1.legend(leg_handles, ['HC', 'SC'], title='')



points = sns.pointplot(
    data=cells.loc[cells['is_HC']],
    ax=ax2,
    x='stage',
    y='normalized_area_cells',
    hue='position',
    hue_order=['25I', '75I'],
    palette='Oranges',
    join=False,
    errorbar='sd',
    # capsize=.05,
    dodge=True,
    errwidth=linewidth
)
plt.setp(points.collections, sizes=[markersize])


leg_handles = ax2.get_legend_handles_labels()[0]
ax2.legend(leg_handles, ['25I', '75I'], title='')


ax2.hlines(y=[1], xmin=-1, xmax=5, ls=':', color='gray')

ax2.set_ylim([0, 5])
ax2.set_xlim([-0.5, 3.5])

ax2.set_ylabel('Normalised HC area')

ax2.set_xlabel('')
ax2.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

plt.savefig(f"{figure_path}/area_development__explants.svg",format='svg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['is_HC']],
    x='stage',
    y='normalized_area_cells',
    hue='position', hue_order=['25I', '75I'],
    # palette=[''],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

print(HC.loc[HC['position'] == '25I'].groupby(by='stage')['normalized_area_cells'].mean())

ax.hlines(y=[1], xmin=-1, xmax=3.5, ls=':', color='gray')


leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['25I', '75I'], title='')


ax.set_ylim([0, 4.5])
ax.set_xlim([-0.5, 3.5])

ax.set_ylabel('Normalised HC area')
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)


plt.savefig(f"{figure_path}/HC_area.svg",format='svg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

points = sns.pointplot(
    data=tissue_data,
    x='stage',
    y='HC_density',
    # hue='position', # hue_order=['25I', '75I'],
    # palette=[''],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
)
points = sns.scatterplot(
    data=tissue_data,
    x='stage',
    y='HC_density',
    # hue='position', # hue_order=['25I', '75I'],
    # palette=[''],
    markers='.',
    alpha=0.4,
)
plt.setp(points.collections, sizes=[markersize])

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['25I', '75I'], title='')

ax.set_ylim([0, 0.4])
ax.set_xlim([-0.5, 3.5])

ax.set_ylabel('HC fraction')
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)


plt.savefig(f"{figure_path}/HC_density__stages.svg",format='svg')

In [ ]:
print(tissue_data.loc[(tissue_data['PD_position']==25) & (tissue_data['stage']=='E14')].to_string())

In [ ]:
for pos in [25, 50, 75, 100]:
    # fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])
    fig, ax = plt.subplots(1, 1, figsize=[2, 1.4])

    points = sns.pointplot(
        data=tissue_data.loc[tissue_data['PD_position']==pos],
        x='stage',
        y='HC_density',
        # hue='position', hue_order=['25I', '25S'],
        # palette=[''],
        join=False,
        errorbar='sd',
        capsize=.05,
        dodge=True,
        errwidth=linewidth,
        markers='_',
        color='black'
    )
    # plt.setp(points.collections, sizes=[markersize])

    points = sns.scatterplot(
        data=tissue_data.loc[tissue_data['PD_position']==pos],
        x='stage',
        y='HC_density',
        markers='.',
        alpha=0.4,
        color='black'
    )
    plt.setp(points.collections, sizes=[markersize])

    box_pairs=[
        ('E10', 'E14'),
        ('E12', 'E14'),
    ]


    ax.set_ylim([0, 0.4])
    ax.set_xlim([-0.5, 3.5])

    ax.set_ylabel('HC fraction')
    ax.set_xlabel('')
    # ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)


    annotator = Annotator(
        ax,
        box_pairs,
        data=tissue_data.loc[tissue_data['PD_position']==pos],
        x='stage',
        y='HC_density',
        split=False,
    )
    annotator.configure(
        test='Mann-Whitney',
        line_width=0.75,
        loc='outside',
        text_format='simple'
    )
    annotator.apply_and_annotate()


    plt.savefig(f"{figure_path}/HC_density__stages__{pos}IS.svg",format='svg')

In [ ]:
print(tissue_data.loc[((tissue_data['PD_position']==25) | (tissue_data['PD_position']==75))].to_string())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

# print(tissue_data.to_string())

points = sns.pointplot(
    data=tissue_data.loc[(tissue_data['PD_position']==25) | (tissue_data['PD_position']==75)],
    x='stage',
    y='HC_density',
    hue='PD_position', hue_order=[25, 75],
    # hue='position', hue_order=['25I', '25S'],
    # palette=[''],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=.25,
    errwidth=linewidth,
    markers='_',
    # markersize=20, markeredgewidth=3,
    color='red'
)
plt.setp(points.collections, sizes=[4*markersize])
points = sns.swarmplot(
    data=tissue_data.loc[(tissue_data['PD_position']==25) | (tissue_data['PD_position']==75)],
    x='stage',
    y='HC_density',
    # hue='position', hue_order=['25I', '25S'],
    hue='PD_position', hue_order=[25, 75],
    # hue='position', # hue_order=['25I', '75I'],
    # palette=[''],
    # markers='.',
    alpha=0.6,
    size=2.,
    color='red'
)
# plt.setp(points.collections, sizes=[markersize])

# leg_handles = ax.get_legend_handles_labels()[0]
# ax.legend(leg_handles, ['25I', '75I'], title='')


box_pairs=[
    # ('E8', 'E10'),
    # ('E8', 'E12'),
    # ('E8', 'E14'),
    # ('E10', 'E12'),
    ('E10', 'E14'),
    ('E12', 'E14'),
]


ax.set_ylim([0, 0.4])
ax.set_xlim([-0.5, 3.5])

ax.set_ylabel('HC fraction')
ax.set_xlabel('')
# ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)


# annotator = Annotator(
#     ax,
#     box_pairs,
#     data=tissue_data.loc[tissue_data['PD_position']==75],
#     x='stage',
#     y='HC_density',
#     split=False,
# )
# annotator.configure(
#     test='Mann-Whitney',
#     line_width=0.75,
#     # loc='outside',
#     text_format='simple'
# )
# annotator.apply_and_annotate()


plt.savefig(f"{figure_path}/HC_density__stages__25_75IS.svg",format='svg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

# print(tissue_data.to_string())

points = sns.pointplot(
    data=tissue_data.loc[(tissue_data['position']=='25S') | (tissue_data['position']=='75S')],
    x='stage',
    y='HC_density',
    hue='PD_position', hue_order=[25, 75],
    # hue='position', hue_order=['25I', '25S'],
    # palette=[''],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=.25,
    errwidth=linewidth,
    markers='_',
    # markersize=20, markeredgewidth=3,
    color='red'
)
plt.setp(points.collections, sizes=[4*markersize])
points = sns.swarmplot(
    data=tissue_data.loc[(tissue_data['position']=='25S') | (tissue_data['position']=='75S')],
    x='stage',
    y='HC_density',
    # hue='position', hue_order=['25I', '25S'],
    hue='PD_position', hue_order=[25, 75],
    # hue='position', # hue_order=['25I', '75I'],
    # palette=[''],
    # markers='.',
    alpha=0.6,
    size=2.,
    color='red'
)
# plt.setp(points.collections, sizes=[markersize])

# leg_handles = ax.get_legend_handles_labels()[0]
# ax.legend(leg_handles, ['25I', '75I'], title='')


box_pairs=[
    # ('E8', 'E10'),
    # ('E8', 'E12'),
    # ('E8', 'E14'),
    # ('E10', 'E12'),
    ('E10', 'E14'),
    ('E12', 'E14'),
]


ax.set_ylim([0, 0.4])
ax.set_xlim([-0.5, 3.5])

ax.set_ylabel('HC fraction')
ax.set_xlabel('')
# ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)


# annotator = Annotator(
#     ax,
#     box_pairs,
#     data=tissue_data.loc[tissue_data['PD_position']==75],
#     x='stage',
#     y='HC_density',
#     split=False,
# )
# annotator.configure(
#     test='Mann-Whitney',
#     line_width=0.75,
#     # loc='outside',
#     text_format='simple'
# )
# annotator.apply_and_annotate()


plt.savefig(f"{figure_path}/HC_density__stages__25S_75S.svg",format='svg')

In [ ]:
print(sns.__version__)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])
fig.tight_layout()

sns.pointplot(
    data=tissue_data.loc[tissue_data['position']=='25I'],
    x='stage',
    y='SC_area__no_norm',
    palette='Oranges',
    color='gray',
    join=False,
    errorbar='sd',
    capsize=.05,
    errwidth=linewidth,
    dodge=True,
    markers='.',
)

sns.pointplot(
    data=tissue_data.loc[tissue_data['position']=='25I'],
    x='stage',
    y='HC_area__no_norm',
    color='red',
    join=False,
    errorbar='sd',
    capsize=.05,
    errwidth=linewidth,
    dodge=True,
    markers='.',
)

ax.set_ylim([0, None])
ax.set_xlim([-0.5, 3.5])
ax.set_xticklabels([8, 10, 12, 14])

ax.set_ylabel('Cell area [$\mu m^2 $]')
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

# Mean of mean
plt.savefig(f"{figure_path}/HC_area__no_norm__tissues.svg",format='svg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

points = sns.pointplot(
    data=HC,
    x='stage',
    y='area',
    hue='position', hue_order=['25I', '75I'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.legend().remove()

box_pairs=[
    ('E8', 'E10'),
    ('E8', 'E12'),
    ('E8', 'E14'),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=HC,
    x='stage',
    y='area',
    split=True,
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75,
)

ax.set_ylim([0, None])

ax.set_ylabel('HC area [$\mu m^2$]')
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

annotator.apply_and_annotate()


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

points = sns.pointplot(
    data=SC,
    x='stage',
    y='area',
    hue='position', hue_order=['25I', '75I'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.legend().remove()

box_pairs=[
    ('E8', 'E10'),
    ('E8', 'E12'),
    ('E8', 'E14'),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=SC,
    x='stage',
    y='area',
    split=True,
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75,
)


ax.set_ylim([0, None])

ax.set_ylabel('Cell area [$\mu m^2$]')
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

annotator.apply_and_annotate()


In [ ]:
area_increase = pd.DataFrame()

area_increase['HC_area__E8'] = HC.loc[HC['stage']=='E8'].groupby(by='position')['normalized_area_cells'].mean()
area_increase['HC_area__E10'] = HC.loc[HC['stage']=='E10'].groupby(by='position')['normalized_area_cells'].mean()
area_increase['HC_area__E12'] = HC.loc[HC['stage']=='E12'].groupby(by='position')['normalized_area_cells'].mean()
area_increase['HC_area__E14'] = HC.loc[HC['stage']=='E14'].groupby(by='position')['normalized_area_cells'].mean()

area_increase['SC_area__E8'] = SC.loc[SC['stage']=='E8'].groupby(by='position')['normalized_area_cells'].mean()
area_increase['SC_area__E10'] = SC.loc[SC['stage']=='E10'].groupby(by='position')['normalized_area_cells'].mean()
area_increase['SC_area__E12'] = SC.loc[SC['stage']=='E12'].groupby(by='position')['normalized_area_cells'].mean()
area_increase['SC_area__E14'] = SC.loc[SC['stage']=='E14'].groupby(by='position')['normalized_area_cells'].mean()

area_increase['ration__E14_E8'] = area_increase['HC_area__E14'] / area_increase['HC_area__E8']
area_increase['ration__E12_E10'] = area_increase['HC_area__E12'] / area_increase['HC_area__E10']
area_increase['ration__E12_E8'] = area_increase['HC_area__E12'] / area_increase['HC_area__E8']


area_increase['std__HC_area__E8'] = HC.loc[HC['stage']=='E8'].groupby(by='position')['normalized_area_cells'].std()
area_increase['std__HC_area__E10'] = HC.loc[HC['stage']=='E10'].groupby(by='position')['normalized_area_cells'].std()
area_increase['std__HC_area__E12'] = HC.loc[HC['stage']=='E12'].groupby(by='position')['normalized_area_cells'].std()
area_increase['std__HC_area__E14'] = HC.loc[HC['stage']=='E14'].groupby(by='position')['normalized_area_cells'].std()

area_increase['std__SC_area__E8'] = SC.loc[SC['stage']=='E8'].groupby(by='position')['normalized_area_cells'].std()
area_increase['std__SC_area__E10'] = SC.loc[SC['stage']=='E10'].groupby(by='position')['normalized_area_cells'].std()
area_increase['std__SC_area__E12'] = SC.loc[SC['stage']=='E12'].groupby(by='position')['normalized_area_cells'].std()
area_increase['std__SC_area__E14'] = SC.loc[SC['stage']=='E14'].groupby(by='position')['normalized_area_cells'].std()

print(area_increase.to_string())

print("No normalisation")
area_increase['HC_area__no_norm__E8'] = HC.loc[HC['stage']=='E8'].groupby(by='position')['area'].mean()
area_increase['HC_area__no_norm__E10'] = HC.loc[HC['stage']=='E10'].groupby(by='position')['area'].mean()
area_increase['HC_area__no_norm__E12'] = HC.loc[HC['stage']=='E12'].groupby(by='position')['area'].mean()
area_increase['HC_area__no_norm__E14'] = HC.loc[HC['stage']=='E14'].groupby(by='position')['area'].mean()
area_increase['ration__no_norm__E14_E8'] = area_increase['HC_area__no_norm__E14'] / area_increase['HC_area__no_norm__E8']
area_increase['ration__no_norm__E12_E10'] = area_increase['HC_area__no_norm__E12'] / area_increase['HC_area__no_norm__E10']
area_increase['ration__no_norm__E12_E8'] = area_increase['HC_area__no_norm__E12'] / area_increase['HC_area__no_norm__E8']

print(area_increase.to_string())

# Tissue geometries

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=[10, 10], sharex=True, sharey='col')
fig.tight_layout()

groupby = [
    'stage',
    'position',
    'file_id',
    'num_neighbors',
]
norm = cells.groupby(by=groupby[:-1])['area'].mean().reset_index()
norm['frequency'] = cells.groupby(by=groupby[:-1]).apply(lambda cells: cells.shape[0]).reset_index().rename(columns=dict({0: 'frequency'}))['frequency']

data = cells.groupby(by=groupby)['area'].mean().reset_index()
data['frequency'] = cells.groupby(by=groupby).apply(lambda cells: cells.shape[0]).reset_index().rename(columns=dict({0: 'frequency'}))['frequency']
data['area'] = data.apply(lambda data, *, norm: data['area'] / norm.loc[norm['file_id']==data['file_id'], 'area'].iloc[0], axis=1, norm=norm)
data['frequency'] = data.apply(lambda data, *, norm: data['frequency']*100 / norm.loc[norm['file_id']==data['file_id'], 'frequency'].iloc[0], axis=1, norm=norm)

drosophila = pd.DataFrame(
    [
        [3, 0.42, 1.00], 
        [4, 0.56, 6.78], 
        [5, 0.82, 34.61],
        [6, 1.08, 38.28],
        [7, 1.36, 14.28],
        [8, 1.52, 2.18]
    ],
    columns=['num_neighbors', 'area', 'frequency']
)
drosophila['stage'] = 'Drosophila'
drosophila['position'] = 'Drosophila'
drosophila['file_id'] = 'Drosophila'

data = pd.concat([data, drosophila])

print(data.groupby(by=['stage', 'position', 'num_neighbors'])['area', 'frequency'].mean().to_string())

ax = axes[0][0]
points = sns.pointplot(
    data=data.loc[((data['position'] == '25I') & (data['stage']=='E8')) | (data['stage']=='Drosophila')],
    x='num_neighbors',
    y='area',
    hue='stage',
    # hue_order=[False, True],
    # palette=['gray', 'red'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
    ax=ax
)

ax = axes[0][1]
points = sns.barplot(
    data=data.loc[((data['position'] == '25I') & (data['stage']=='E8')) | (data['stage']=='Drosophila')],
    x='num_neighbors',
    y='frequency',
    hue='stage',
    # hue_order=[False, True],
    # palette=['gray', 'red'],
    dodge=True,
    ax=ax
)



ax = axes[1][0]
points = sns.pointplot(
    data=data.loc[((data['position'] == '25I') & (data['stage']=='E10')) | (data['stage']=='Drosophila')],
    x='num_neighbors',
    y='area',
    hue='stage',
    # hue_order=[False, True],
    # palette=['gray', 'red'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
    ax=ax
)

ax = axes[1][1]
points = sns.barplot(
    data=data.loc[((data['position'] == '25I') & (data['stage']=='E10')) | (data['stage']=='Drosophila')],
    x='num_neighbors',
    y='frequency',
    hue='stage',
    # hue_order=[False, True],
    # palette=['gray', 'red'],
    dodge=True,
    ax=ax
)



ax = axes[2][0]
points = sns.pointplot(
    data=data.loc[((data['position'] == '25I') & (data['stage']=='E12')) | (data['stage']=='Drosophila')],
    x='num_neighbors',
    y='area',
    hue='stage',
    # hue_order=[False, True],
    # palette=['gray', 'red'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
    ax=ax
)

ax = axes[2][1]
points = sns.barplot(
    data=data.loc[((data['position'] == '25I') & (data['stage']=='E12')) | (data['stage']=='Drosophila')],
    x='num_neighbors',
    y='frequency',
    hue='stage',
    # hue_order=[False, True],
    # palette=['gray', 'red'],
    dodge=True,
    ax=ax
)



ax = axes[3][0]
points = sns.pointplot(
    data=data.loc[((data['position'] == '25I') & (data['stage']=='E14')) | (data['stage']=='Drosophila')],
    x='num_neighbors',
    y='area',
    hue='stage',
    # hue_order=[False, True],
    # palette=['gray', 'red'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
    ax=ax
)

ax = axes[3][1]
points = sns.barplot(
    data=data.loc[((data['position'] == '25I') & (data['stage']=='E14')) | (data['stage']=='Drosophila')],
    x='num_neighbors',
    y='frequency',
    hue='stage',
    # hue_order=[False, True],
    # palette=['gray', 'red'],
    dodge=True,
    ax=ax
)

fig.savefig(f"{figure_path}/tissue_characterisation.svg")

## Neighbourhood

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/5, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['position'] == '25I'],
    x='stage',
    y='num_neighbors',
    hue='is_HC',
    hue_order=[False, True],
    palette=['gray', 'red'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])


ax.set_ylim([3, None])
ax.set_xlim([-0.5, 3.5])

ax.set_ylabel('Neighbour number')

ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['SC', 'HC'], title='')

plt.savefig(f"{figure_path}/num_neighbors.svg",format='svg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['position']=='25I'],
    x='stage',
    y='num_neighbors',
    hue='is_HC',
    hue_order=[False, True],
    palette=['gray', 'red'],
    color='red',
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.set_ylim([3, 11])
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(1))

ax.set_xlim([-0.5, 3.5])
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

ax.set_ylabel('Neighbour number')
# ax.set_yticks([0., 0.5, 1.])

ax.hlines(y=[6], xmin=-0.5, xmax=3.5, colors='black', lw=linewidth)

plt.savefig(f"{figure_path}/num_neighbors__25I.svg")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/5, 5.9/3])

points = sns.pointplot(
    data=HC,
    x='stage',
    y='num_neighbors',
    hue='position', hue_order=['25I', '75I'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])


leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['25I', '75I'], title='')


ax.set_ylim([3, 11])
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)


plt.savefig(f"{figure_path}/HC_neighbour_number.svg",format='svg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['position']=='25I'],
    x='stage',
    y='num_hair_neighbors',
    hue='is_HC',
    hue_order=[False, True],
    palette=['gray', 'red'],
    color='red',
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)
ax.set_xlim([-0.5, 3.5])

ax.set_ylabel('HC neighbours')

ax.set_ylim([-.1, 2.5])
ax.set_yticks([0, 1, 2, 3])
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.5))

ax.hlines(y=[0], xmin=-0.5, xmax=3.5, colors='black', lw=linewidth, zorder=-np.inf)

plt.savefig(f"{figure_path}/hair_neighbour_numbers_25I.svg")

### Neighbourhood histogram plots

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

mask = np.where(
    cells['HC_normalized_area']
)

data = cells.loc[(cells['stage'] == 'E10') & (cells['position'] == '25I')]
print(data['filename'].unique())

sns.histplot(
    data=data,
    x='num_neighbors',
    binrange=[3, 12],
    discrete=True,
    hue='is_HC',
    palette=['gray', 'red'],
    multiple='stack',
    stat='probability',
    common_norm=True,
    legend=False,
    alpha=0.5
)

__d = data
__hc = __d.loc[__d['is_HC']]
ax.vlines(
    x=[
        __d['num_neighbors'].mean(),
        __d.loc[__d['is_HC'], 'num_neighbors'].mean(),
        __d.loc[~__d['is_HC'], 'num_neighbors'].mean(),
    ],
    ymin=[
        0,
        0,
        __hc.loc[__hc['num_neighbors']== 6, 'num_neighbors'].count() / __d['num_neighbors'].count(),
    ],
    ymax=[
        __d.loc[__d['num_neighbors']== 6, 'num_neighbors'].count() / __d['num_neighbors'].count(),
        __hc.loc[__hc['num_neighbors']==5, 'num_neighbors'].count() / __d['num_neighbors'].count(),
        __d.loc[__d['num_neighbors']== 6, 'num_neighbors'].count() / __d['num_neighbors'].count(),
    ],
    colors=['black', 'red', 'gray'],
    lw=0.5,
    ls='dashed',
)

print([
        __d['num_neighbors'].mean(),
        __d.loc[__d['is_HC'], 'num_neighbors'].mean(),
        __d.loc[~__d['is_HC'], 'num_neighbors'].mean(),
    ])

ax.set_xlim([2.5, 10.5])
ax.set_xticks([4, 6, 8, 10])
ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

ax.set_ylim([0, 0.4])

ax.set_xlabel('Neighbour number')
ax.set_ylabel('Frequency')


plt.savefig(f"{figure_path}/neighbours_hc_vs_sc_stack.svg")

print(
    "normalised area:",
    "%.2f"%__d.loc[__d['is_HC'] == True, 'normalized_area_cells'].mean(),
    "pm %.2f"%__d.loc[__d['is_HC'] == True, 'normalized_area_cells'].std(),
    ". %.2f"%__d.loc[__d['is_HC'] == False, 'normalized_area_cells'].mean(),
    "pm %.2f"%__d.loc[__d['is_HC'] == False, 'normalized_area_cells'].std(),
)
print(
    "num neighbours:",
    "%.2f"%__d.loc[__d['is_HC'] == True, 'num_neighbors'].mean(),
    "pm %.2f"%__d.loc[__d['is_HC'] == True, 'num_neighbors'].std(),
    ". %.2f"%__d.loc[__d['is_HC'] == False, 'num_neighbors'].mean(),
    "pm %.2f"%__d.loc[__d['is_HC'] == False, 'num_neighbors'].std(),
)
print("Welch's t-test on neighbourhood:",
    "%.2E" % scipy.stats.ttest_ind(
        __d.loc[__d['is_HC'] == True, 'num_neighbors'],
        __d.loc[__d['is_HC'] == False, 'num_neighbors'],
        equal_var=False
    )[1]
)
print("Welch's t-test on area:",
    "%.2E" % scipy.stats.ttest_ind(
        __d.loc[__d['is_HC'] == True, 'normalized_area_cells'],
        __d.loc[__d['is_HC'] == False, 'normalized_area_cells'],
        equal_var=False
    )[1]
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

mask = np.where(
    cells['HC_normalized_area']
)

data = cells.loc[(cells['stage'] == 'E10') & (cells['position'] == '25I')]

bins = [
    2.6, 3.4,
    3.6, 4.4,
    4.6, 5.4,
    5.6, 6.4,
    6.6, 7.4,
    7.6, 8.4,
    8.6, 9.4,
    9.6, 10.4,
    10.6
]

sns.histplot(
    data=data,
    x='num_neighbors',
    # binrange=[3, 12],
    bins=bins,
    # discrete=True,
    hue='is_HC',
    palette=['gray', 'red'],
    multiple='dodge',
    # multiple='stack',
    stat='probability',
    common_norm=False,
    legend=False,
    alpha=0.5
)

ax.set_xlim([2.5, 10.5])
ax.set_xticks([4, 6, 8, 10])
ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

# ax.set_ylim([0, 3050])
ax.set_ylim([0, 0.5])
ax.set_yticks([0., 0.2, 0.4])
ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

ax.set_xlabel('Neighbour number')
ax.set_ylabel('Frequency')


plt.savefig(f"{figure_path}/neighbours_hc_vs_sc_dodge.svg")


fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

sns.histplot(
    data=data,
    x='num_neighbors',
    binrange=[3, 12],
    # bins=bins,
    discrete=True,
    hue='is_HC',
    palette=['gray', 'red'],
    # multiple='dodge',
    # multiple='stack',
    stat='probability',
    common_norm=False,
    legend=False,
    alpha=0.5
)

ax.set_xlim([2.5, 10.5])
ax.set_xticks([4, 6, 8, 10])
ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

# ax.set_ylim([0, 3050])
ax.set_ylim([0, 0.5])
ax.set_yticks([0., 0.2, 0.4])
ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

ax.set_xlabel('Neighbour number')
ax.set_ylabel('Frequency')

print(data.loc[data['is_HC'], 'num_neighbors'].mean())
print(data.loc[~data['is_HC'], 'num_neighbors'].mean())


plt.savefig(f"{figure_path}/neighbours_hc_vs_sc_layer.svg")

In [ ]:
def x_to_xtick(x, *, x_ticks):
    for i in range(len(x_ticks)):
        if x == x_ticks[i]:
            return 2 * i + 1
    raise

def histplot_violin(
    hist, *, 
    data_column, ax, label, 
    x_ticks, x_tick_labels, y_ticks, y_tick_labels, 
    set_colors, set_background=[],
    means=None,
    hist_split=None,
    means_split=None,
    split_color='green'
):
    ax.set_xlim([0, 2*len(x_ticks)])
    ax.set_xticks(np.arange(1, 2*len(x_ticks)+1, 2), x_tick_labels, rotation = 30)
    ax.tick_params(axis='x', which='major', pad=0)
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(2))
    for label in ax.get_xmajorticklabels():
        label.set_horizontalalignment("right")
    
    ax.set_yticks(
        np.arange(
            y_ticks[0],
            y_ticks[-1]+1,
            round((y_ticks[-1])/(len(y_tick_labels)-1))
        ),
        y_tick_labels
        )
    ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(1))
    # ax.set_aspect(0.8)

    boxes = []
    colors = []
    for i, color in enumerate(set_background):
        if len(color) == 0:
            continue

        boxes.append(
            Rectangle(
                (2*i, 0),
                width=2,
                height=len(y_ticks),
                ls='',
                lw=None,
            )
        )
        colors.append(color)
    ax.add_collection(
        PatchCollection(
            boxes,
            facecolor=colors,
            edgecolor='None',
            alpha=0.2,
        )
    )

    boxes = []
    colors = []
    for i, (xtick, color) in enumerate(zip(x_ticks, set_colors)):
        _d = hist.loc[hist[data_column] == xtick]
        if hist_split is not None:
            split_d = hist_split.loc[hist_split[data_column] == xtick]

        for N in y_ticks:
            locx = x_to_xtick(xtick, x_ticks=x_ticks)
            count = _d[N].iloc[0]
            if np.isnan(count):
                count = 0

            boxes.append(
                Rectangle(
                    (locx, N - 0.5),
                    width=-count,
                    height=1,
                    ls='',
                    lw=0,
                )
            )
            colors.append(color)
            
            if hist_split is None:
                boxes.append(
                    Rectangle(
                        (locx, N - 0.5),
                        width=count,
                        height=1,
                        ls='',
                        lw=0,
                    )
                )
                colors.append(color)
            else:
                count = split_d[N].iloc[0]
                boxes.append(
                    Rectangle(
                        (locx, N - 0.5),
                        width=count,
                        height=1,
                        ls='',
                        lw=0,
                    )
                )
                colors.append(split_color)
        
        # plot mean as hline
        if means is not None and len(means):
            xtick = x_ticks[i]
            mean = means[i]
            count = _d[round(mean)].iloc[0]
            if means_split is None:
                xmax = 2*i + 1 + count
            else:
                xmax = 2*i + 1

            ax.hlines(
                y=mean,
                xmin=2*i + 1 - count,
                xmax=xmax,
                color='black',
                linestyles='dashed',
                linewidths=linewidth
            )

        if means_split is not None and len(means_split):
            xtick = x_ticks[i]
            mean = means_split[i]
            count = split_d[round(mean)].iloc[0]
            ax.hlines(
                y=mean,
                xmin=2*i + 1,
                xmax=2*i + 1 + count,
                color='black',
                linestyles='dashed',
                linewidths=linewidth
            )


    pc = PatchCollection(boxes, facecolor=colors)
    histogram = ax.add_collection(pc)
    # legend_elements = [
    #     mpl.patches.Patch(label='stage: {}'.format(stage)),
    #     # mpl.patches.Patch(facecolor='orange', edgecolor='orange', label='Theory')
    # ]
    # if label is not None:
    #     print(label)
    #     if type(label) is not str:
    #         label = 'E14'
    #     # NOTE weird error
    #     ax.set_title('{:s}'.format(str(label)), x=0.85, y=0.8, fontsize='small')
    # ax.legend(title='stage: {}'.format(stage), loc='upper right')
    # ax.legend(handles=legend_elements, loc='upper right')
    ax.set_ylabel('Neighbour numbers')
    ax.set_ylim(0, len(y_ticks)-1)


def histplot_violin_col(*, hist, col_labels, col_axes, col_column, means=None, means_split=None, **kwargs):
    hist = hist.reset_index()
    if means_split is None:
        means_split = []
        for a in col_axes:
            means_split.append(None)
    if means is None:
        means = []
        for a in col_axes:
            means.append(None)
    for (col, ax, _means, _means_split) in zip(col_labels, col_axes, means, means_split):
        histplot_violin(
            hist.loc[hist[col_column] == col], 
            ax=ax,
            label=col, 
            means=_means, means_split=_means_split,
            **kwargs
        )


In [ ]:
for position in ['25I', '50I', '75I']:
    figsize=[6.7, 1.9]
    fig, axes = plt.subplots(1, 4, figsize=figsize, sharex=True, sharey=True)
    (ax1, ax2, ax3, ax4) = axes

    for ax in axes:
        for axis in ['bottom','left']:
            ax.spines[axis].set_linewidth(1./4)
        for axis in ['top','right']:
            ax.spines[axis].set_linewidth(0)
        ax.tick_params(which='both', width=1./4)

    stages = [
        'E8',
        'E10',
        'E12',
        'E14',
    ]

    y_min = 0
    y_max=12
    y_ticks = np.arange(0, y_max+1)
    y_tick_labels = [
        '0',# '',
        '3',# '',
        '6',# '',
        '9',# '',
        '12',
    ]

    data = cells.loc[cells['position'] == position]

    hc = data.loc[data['is_HC']].copy()
    sc = data.loc[~data['is_HC']].copy()


    # HC neighborhood
    hc_norm = hc.groupby(by='stage')['num_neighbors'].count()

    _hist = [hc.loc[hc['num_neighbors'] == i].groupby(by='stage')['num_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(hc_norm, axis=0)
    _hist['type'] = 'H'

    hist = _hist

    ### HH
    _hist = [hc.loc[hc['num_hair_neighbors'] == i].groupby(by='stage')['num_hair_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(hc_norm, axis=0)
    _hist['type'] = 'HH'

    hist = pd.concat([hist, _hist], axis=0, join='outer')

    ### HS
    hc['num_support_neighbors'] = hc['num_neighbors'] - hc['num_hair_neighbors']
    _hist = [hc.loc[hc['num_support_neighbors'] == i].groupby(by='stage')['num_support_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(hc_norm, axis=0)
    _hist['type'] = 'HS'

    hist = pd.concat([hist, _hist], axis=0, join='outer')


    # SC neighborhood
    sc_norm = sc.groupby(by='stage')['num_neighbors'].count()

    _hist = [sc.loc[sc['num_neighbors'] == i].groupby(by='stage')['num_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(sc_norm, axis=0)
    _hist['type'] = 'S'

    hist = pd.concat([hist, _hist], axis=0, join='outer')

    # SH
    _hist = [sc.loc[sc['num_hair_neighbors'] == i].groupby(by='stage')['num_hair_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(sc_norm, axis=0)
    _hist['type'] = 'SH'

    hist = pd.concat([hist, _hist], axis=0, join='outer')

    # SS
    sc['num_support_neighbors'] = sc['num_neighbors'] - sc['num_hair_neighbors']
    _hist = [sc.loc[sc['num_support_neighbors'] == i].groupby(by='stage')['num_support_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(sc_norm, axis=0)
    _hist['type'] = 'SS'


    # concat
    hist = pd.concat([hist, _hist], axis=0, join='outer')
    hist = hist.reset_index()
    hist = hist.set_index(['stage', 'type'])

    hist = hist.reset_index()


    # fig, axes = plt.subplots(1, 4, figsize=figsize, sharex=True, sharey=True)
    # (ax1, ax2, ax3, ax4) = axes

    for ax in axes:
        for axis in ['bottom','left']:
            ax.spines[axis].set_linewidth(1./4)
        for axis in ['top','right']:
            ax.spines[axis].set_linewidth(0)
        ax.tick_params(which='both', width=1./4)

    x_ticks = [
        'H',
        'HH',
        # 'HS',
        'S',
        'SH',
        'SS'
    ]
    x_tick_labels = [
        'HC (all)',
        'HC-HC',
        #'', 'HC-SC',
        'SC (all)', 
        'SC-HC',
        'SC-SC',
    ]


    set_colors = [
        'red',
        'orangered',
        # 'gold',
        'dimgrey',
        'darkgray',
        'silver'
    ]

    set_background = [
        # 'coral',
        # '',
        # 'gray',
    ]

    means = []

    for stage in ['E8', 'E10', 'E12', 'E14']:
        mean_data = data.loc[data['stage'] == stage]
        tmp = [
            mean_data.loc[mean_data['is_HC']]['num_neighbors'].mean(),
            mean_data.loc[mean_data['is_HC']]['num_hair_neighbors'].mean(),
            mean_data.loc[~mean_data['is_HC']]['num_neighbors'].mean(),
            mean_data.loc[~mean_data['is_HC']]['num_hair_neighbors'].mean(),
            mean_data.loc[~mean_data['is_HC']]['num_neighbors'].mean() - mean_data.loc[~mean_data['is_HC']]['num_hair_neighbors'].mean(),
        ]
        means.append(tmp)

    histplot_violin_col(
        hist=hist,
        col_column='stage',
        col_labels=stages,
        col_axes=[ax1, ax2, ax3, ax4],
        x_ticks=x_ticks,
        x_tick_labels=x_tick_labels,
        y_ticks=y_ticks,
        y_tick_labels=y_tick_labels,
        set_colors=set_colors,
        data_column='type',
        set_background=set_background,
        means=means,
    )


    fig.tight_layout()

    plt.savefig(f"{figure_path}/neighborhood__{position}.svg")


### Neighbourhood perturbation histograms

In [ ]:
def plot_hist(*, data, x, discrete_bins=False,  **kwargs):
    ax = plt.gca()

    sns.histplot(
        data=data,
        x=x,
        hue='is_HC',
        palette=['gray', 'red'],
        multiple='stack',
        stat='probability',
        # common_norm=True,
        legend=False,
        alpha=0.5,
        **kwargs
    )

    __hc = data.loc[data['is_HC']]
    hc_mean = data.loc[data['is_HC'], x].mean()
    sc_mean = data.loc[~data['is_HC'], x].mean()

    vlines_kwargs = dict({
        'lw': linewidth,
        'ls': 'dashed',
        'colors': ['black', 'red', 'gray'],
    })

    if discrete_bins:
        __mean = round(data[x].mean())
        __hc_mean = round(hc_mean)
        __sc_mean = round(sc_mean)

        plt.vlines(
            x=[
                data[x].mean(),
                hc_mean,
                sc_mean,
            ],
            ymin=[
                0,
                0,
                __hc.loc[np.round(__hc[x])==np.round(__sc_mean), x].count() / data[x].count(),
            ],
            ymax=[
                data.loc[np.round(data[x])==np.round(__mean), x].count() / data[x].count(),
                __hc.loc[np.round(__hc[x])==np.round(__hc_mean), x].count() / data[x].count(),
                data.loc[np.round(data[x])==np.round(__sc_mean), x].count() / data[x].count(),
            ],
            **vlines_kwargs
        )
    else:
        ymax = ax.get_ylim()[1]
        plt.vlines(
            x=[
                data[x].mean(),
                hc_mean,
                sc_mean,
            ],
            ymin=[
                0,
                0,
                0.,
            ],
            ymax=[
                ymax,
                ymax,
                ymax
            ],
            **vlines_kwargs
        )

        

    legend = ax.legend(
        labels=['all','HC', 'SC'],
    )

In [ ]:
treatments = [
    'Control',

    'MLCKdmso-4hrs',
    'MLCKdmso-16hrs',
    'MLCKml7-4hrs',
    'MLCKml7-16hrs',

    'Vangl2Control',
    'Vangl2KO',

    'Atoh1control',
    'Atoh1KO',

    'ROCKcontrol-4hrs',
    'ROCKcontrol-16hrs',
    'ROCKy27632-4hrs',
    'ROCKy27632-16hrs',

    'PTXcontrol',
    'PTXinhibition',
]
descriptions = [
    'Fixed BP (Control, 25I @ E10)',

    'MLCK (DMSO, 4hrs)',
    'MLCK (DMSO, 16hrs)',
    'MLCK (4hrs)',
    'MLCK (washoff, 16hrs)',

    'Vangl2 (Control)',
    'Vangl2-KO',

    'Atoh1 (Control)',
    'Atoh1-KO',

    'ROCK (DMSO, 4hrs)',
    'ROCK (DMSO, 16hrs)',
    'ROCK (4hrs)',
    'ROCK (washoff, 16hrs)',

    'PTX (DMSO)',
    'PTX (4hrs)',
]

bins = [
    2.6, 3.4,
    3.6, 4.4,
    4.6, 5.4,
    5.6, 6.4,
    6.6, 7.4,
    7.6, 8.4,
    8.6, 9.4,
    9.6, 10.4,
    10.6
]

try: 
    os.mkdir(f"{figure_path}/neighbors_hc_vs_sc")
except OSError as error: 
    print(error)
try: 
    os.mkdir(f"{figure_path}/neighbors_hh_contacts")
except OSError as error: 
    print(error)
try: 
    os.mkdir(f"{figure_path}/neighbors_hc")
except OSError as error: 
    print(error)


for treatment, description in zip(treatments, descriptions):
    print(treatment)
    data = cells_experiments.loc[cells_experiments['Treatment'] == treatment].copy()
    data = data.loc[data['stage']=='E10']
    data = data.loc[data['position']=='25I']

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[2*5.9/3,5.9/4*3/2])

    plt.sca(ax1)
    plot_hist(data=data, x='normalized_area_cells', discrete = False)
    ax1.set_xlim(0, 3)

    plt.sca(ax2)
    plot_hist(data=data, x=f'num_neighbors', discrete = True, discrete_bins=True)
    ax2.set_xlim(3, 10)

    ax2.set_xlim([3, 10])
    ax2.set_xticks([4, 6, 8, 10])
    ax2.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
    ax2.set_xlabel('Neighbour number')

    plt.suptitle(f'{description}')

    plt.savefig(f"{figure_path}/neighbors_hc_vs_sc/{treatment}.svg")
    ax1.clear()
    ax2.clear()
    plt.close()

    
    fig, ax = plt.subplots(1, 1, figsize=[5.9/4,5.9/4*3/2])

    plt.sca(ax)
    sns.histplot(
        data=data,
        x='num_neighbors',
        bins=bins,
        # discrete=True,
        hue='is_HC',
        palette=['gray', 'red'],
        multiple='dodge',
        stat='probability',
        common_norm=False,
        legend=False,
        alpha=0.5
    )
    ax.set_xlim([2.5, 10])
    ax.set_xticks([4, 6, 8, 10])
    ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
    ax.set_xlabel('Neighbour number')

    ax.set_ylim([0, 0.5])
    ax.set_yticks([0, 0.2, 0.4])
    ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

    plt.suptitle(f'{description}')

    plt.savefig(f"{figure_path}/neighbors_hc_vs_sc/{treatment}__dodge.svg")
    plt.close()

    fig, ax = plt.subplots(1, 1, figsize=[5.9/4,5.9/4*3/2])


    plot_hist(data=data, x=f'num_neighbors__corr', discrete=False, discrete_bins=True, bins=np.arange(2.5, 10.5, 1))
    ax.set_xlim(3, 10)

    ax.set_xlim([3, 10])
    ax.set_xticks([4, 6, 8, 10])
    # ax.set_xticks([3, 6, 9])
    ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
    ax.set_xlabel('corr. Neighbour number')

    ax.set_ylim([0, 0.4])
    ax.set_yticks([0., 0.2, 0.4])

    plt.suptitle(f'{description}')
    plt.savefig(f"{figure_path}/neighbors_hc_vs_sc/{treatment}__corrected.svg")
    plt.close()


    fig, ax = plt.subplots(1, 1, figsize=[5.9/4,5.9/4*3/2])

    sns.histplot(
        data=data.loc[data['is_HC']],
        x='num_hair_neighbors',
        bins=[
            -0.4, 0.4,
            0.6, 1.4,
            1.6, 2.4,
            2.6, 3.4,
            3.6, 4.4
        ],
        hue='is_HC',
        palette=['red'],
        multiple='dodge',
        stat='probability',
        common_norm=False,
        legend=False,
        alpha=0.5
    )

    ax.set_xlim([-0.5, 4.5])
    ax.set_xticks([0, 1, 2, 3, 4])
    ax.set_xlabel('Neighbour number')

    ax.set_ylim([0, 1.])
    ax.set_yticks([0., 0.5, 1.])
    ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

    plt.suptitle(f'{description}')
    plt.savefig(f"{figure_path}/neighbors_hh_contacts/{treatment}.svg")
    plt.close()

In [ ]:
figsizes=dict({
    'Vangl2': [1.6, 1.475],
    'MLCK': [1.3, 1.3],
    'ROCK': [1.4, 1.2],
    'ROCK_16hrs': [1.4, 1.2],
    'MLCK_16hrs': [1.4, 1.2],
    'PTX': [1.6, 1.4]
})

data = cells_experiments.copy()

tmp = data.loc[(data['Experiment_Series'] == 'MLCK') & (~data['is_Control'])].copy()
tmp['Experiment'] = 'MLCK_inhibition'
tmp['Experiment_Series'] = 'MLCK_recovery'
tmp['is_Control'] = True
data = pd.concat([data, tmp])

tmp = data.loc[(data['Experiment_Series'] == 'MLCK_16hrs') & (~data['is_Control'])].copy()
tmp['Experiment'] = 'MLCK_recovery'
tmp['Experiment_Series'] = 'MLCK_recovery'
tmp['is_Control'] = False
data = pd.concat([data, tmp])

tmp = data.loc[(data['Experiment_Series'] == 'ROCK') & (~data['is_Control'])].copy()
tmp['Experiment'] = 'ROCK_inhibition'
tmp['Experiment_Series'] = 'ROCK_recovery'
tmp['is_Control'] = True
data = pd.concat([data, tmp])

tmp = data.loc[(data['Experiment_Series'] == 'ROCK_16hrs') & (~data['is_Control'])].copy()
tmp['Experiment'] = 'ROCK_recovery'
tmp['Experiment_Series'] = 'ROCK_recovery'

tmp['is_Control'] = False
data = pd.concat([data, tmp])


y_min = 0
y_max=9
y_ticks = np.arange(0, y_max+1)
y_tick_labels = [
    '0',# '',
    '3',# '',
    '6',# '',
    '9',# '',
    # '12',
]

try: 
    os.mkdir(f"{figure_path}/neighborhood")
except OSError as error: 
    print(error)   

for Series in data['Experiment_Series'].unique():
    if Series == 'Control':
        continue

    print("\n")
    print(Series)

    for num_neighbors, description in zip(['num_neighbors', 'num_neighbors__corr'], ['', '__corrected']):
        if Series in figsizes:
            figsize=figsizes[Series]
        else:
            figsize=[2., 2.]
        fig, ax = plt.subplots(1, 1, figsize=figsize, sharex=True, sharey=True)

        for axis in ['bottom','left']:
            ax.spines[axis].set_linewidth(1./4)
        for axis in ['top','right']:
            ax.spines[axis].set_linewidth(0)
        ax.tick_params(which='both', width=1./4)


        histograms = dict()
        means = dict()
        for is_Control in [True, False]:
            d = data.loc[
                # (data['position'] == position) &
                # (data['stage'] == stage) &
                (data['Experiment_Series'] == Series) &
                (data['is_Control'] == is_Control)
            ]

            hc = d.loc[d['is_HC']].copy()
            sc = d.loc[~d['is_HC']].copy()


            # HC neighborhood
            hc_norm = hc.groupby(by='stage')[num_neighbors].count()

            _hist = [hc.loc[(hc[num_neighbors] > i - 0.5) & (hc[num_neighbors] < i + 0.5)].groupby(by='stage')[num_neighbors].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(hc_norm, axis=0)
            _hist['type'] = 'H'

            hist = _hist

            ### HH
            _hist = [hc.loc[hc['num_hair_neighbors'] == i].groupby(by='stage')['num_hair_neighbors'].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(hc_norm, axis=0)
            _hist['type'] = 'HH'

            hist = pd.concat([hist, _hist], axis=0, join='outer')

            ### HS
            hc['num_support_neighbors'] = hc[num_neighbors] - hc['num_hair_neighbors']
            _hist = [hc.loc[(hc['num_support_neighbors'] > i - 0.5) & (hc['num_support_neighbors'] < i + 0.5)].groupby(by='stage')['num_support_neighbors'].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(hc_norm, axis=0)
            _hist['type'] = 'HS'

            hist = pd.concat([hist, _hist], axis=0, join='outer')


            # SC neighborhood
            sc_norm = sc.groupby(by='stage')[num_neighbors].count()

            _hist = [sc.loc[(sc[num_neighbors] > i - 0.5) & (sc[num_neighbors] < i + 0.5)].groupby(by='stage')[num_neighbors].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(sc_norm, axis=0)
            _hist['type'] = 'S'

            hist = pd.concat([hist, _hist], axis=0, join='outer')

            # SH
            _hist = [sc.loc[sc['num_hair_neighbors'] == i].groupby(by='stage')['num_hair_neighbors'].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(sc_norm, axis=0)
            _hist['type'] = 'SH'

            hist = pd.concat([hist, _hist], axis=0, join='outer')

            # SS
            sc['num_support_neighbors'] = sc[num_neighbors] - sc['num_hair_neighbors']
            _hist = [sc.loc[(sc['num_support_neighbors'] > i - 0.5) & (sc['num_support_neighbors'] < i + 0.5)].groupby(by='stage')['num_support_neighbors'].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(sc_norm, axis=0)
            _hist['type'] = 'SS'


            # concat
            hist = pd.concat([hist, _hist], axis=0, join='outer')
            hist = hist.reset_index()
            hist = hist.set_index(['stage', 'type'])

            hist = hist.reset_index()
            histograms[is_Control] = hist

            means[is_Control] = [[
                hc[num_neighbors].mean(),
                hc['num_hair_neighbors'].mean(),
                sc[num_neighbors].mean(),
                sc['num_hair_neighbors'].mean(),
                sc[num_neighbors].mean() - sc['num_hair_neighbors'].mean(),
            ]]

        x_ticks = [
        'H',
        'HH',
        # 'HS',
        'S',
        'SH',
        'SS'
        ]
        x_tick_labels = [
        'HC (all)',
        'HC-HC',
        #'', 'HC-SC',
        'SC (all)', 
        'SC-HC',
        'SC-SC',
        ]


        set_colors = [
        'red',
        'orangered',
        # 'gold',
        'dimgrey',
        'darkgray',
        'silver'
        ]

        set_background = [
        # 'coral',
        # '',
        # 'gray',
        ]

        histplot_violin_col(
            hist=histograms[True],
            col_column='stage',
            col_labels=['E10'],
            col_axes=[ax],
            x_ticks=x_ticks,
            x_tick_labels=x_tick_labels,
            y_ticks=y_ticks,
            y_tick_labels=y_tick_labels,
            set_colors=set_colors,
            data_column='type',
            set_background=set_background,
            hist_split=histograms[False],
            means=means[True],
            means_split=means[False],
            split_color='blue',
        )

        if len(description) == 0:
            print("HC-HC contacts in ", Series, ": p =",
                "%.2E" % scipy.stats.ttest_ind(
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == True) &
                        (data['is_HC']),
                        'num_hair_neighbors'
                    ],
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == False) &
                        (data['is_HC']),
                        'num_hair_neighbors'
                    ],
                    equal_var=False
                )[1],
                "(",
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == True) &
                    (data['is_HC']),
                    'num_hair_neighbors'
                ].sum(),
                "vs",
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == False) &
                    (data['is_HC']),
                    'num_hair_neighbors'
                ].sum(), 
                ")\n"
            )
            print("number of HC-HC contacts")
            print(
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_HC']),
                ].groupby(by=['is_Control', 'num_hair_neighbors'])['num_hair_neighbors'].count())
            print()
        else:
            print("SC-HC neighbour distributions in ", Series, "(", description, "): p =",
                "%.2E" % scipy.stats.ttest_ind(
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == True) &
                        (data['is_HC'] == False),
                        'num_hair_neighbors'
                    ],
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == False) &
                        (data['is_HC'] == False),
                        'num_hair_neighbors'
                    ],
                    equal_var=True
                )[1]
            )
            print("SC-SC neighbour distributions in ", Series, "(", description, "): p =",
                "%.2E" % scipy.stats.ttest_ind(
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == True) &
                        (data['is_HC'] == False),
                        num_neighbors
                    ]
                    -
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == True) &
                        (data['is_HC'] == False),
                        'num_hair_neighbors'
                    ],
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == False) &
                        (data['is_HC'] == False),
                        num_neighbors
                    ]
                    -
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == False) &
                        (data['is_HC'] == False),
                        'num_hair_neighbors'
                    ],
                    equal_var=True
                )[1]
            )
        print("HC neighbour distributions in ", Series, "(", description, "): p =",
            "%.2E" % scipy.stats.ttest_ind(
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == True) &
                    (data['is_HC']),
                    num_neighbors
                ],
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == False) &
                    (data['is_HC']),
                    num_neighbors
                ],
                equal_var=True
            )[1]
        )
        print("SC neighbour distributions in ", Series, "(", description, "): p =",
            "%.2E" % scipy.stats.ttest_ind(
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == True) &
                    (data['is_HC'] == False),
                    num_neighbors
                ],# ax.set_ylim([0, 3050])

                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == False) &
                    (data['is_HC'] == False),
                    num_neighbors
                ],
                equal_var=True
            )[1]
        )
        print(f"HC vs SC distribution in {Series} control ({description}): p =",
            "%.2E" % scipy.stats.ttest_ind(
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == True) &
                    (data['is_HC'] == True),
                    num_neighbors
                ],
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == True) &
                    (data['is_HC'] == False),
                    num_neighbors
                ],
                equal_var=True
            )[1]
        )
        print(f"HC vs SC distribution in {Series} ({description}): p =",
            "%.2E" % scipy.stats.ttest_ind(
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == False) &
                    (data['is_HC'] == True),
                    num_neighbors
                ],
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == False) &
                    (data['is_HC'] == False),
                    num_neighbors
                ],
                equal_var=True
            )[1]
        )

        plt.savefig(f"{figure_path}/neighborhood/{Series}{description}.svg")
        plt.close()


In [ ]:
Series = 'Atoh1'
print(np.unique(data['Experiment_Series']))
print(data.loc[
                (data['Experiment_Series'] == Series) &
                (data['is_Control'] == True) &
                (data['is_HC'] == False),
                'num_hair_neighbors'
            ])
print(data.loc[
                (data['Experiment_Series'] == Series) &
                (data['is_Control'] == False) &
                (data['is_HC'] == False),
                'num_hair_neighbors'
            ])
print("SC-HC neighbour distributions in ", Series, "(", description, "): p =",
        "%.2E" % scipy.stats.ttest_ind(
            data.loc[
                (data['Experiment_Series'] == Series) &
                (data['is_Control'] == True) &
                (data['is_HC'] == False),
                'num_hair_neighbors'
            ],
            data.loc[
                (data['Experiment_Series'] == Series) &
                (data['is_Control'] == False) &
                (data['is_HC'] == False),
                'num_hair_neighbors'
            ],
            # equal_var=True
        )[1]
    )

In [ ]:
for Series in ['MLCK_recovery', 'ROCK_recovery']:
    fig, ax = plt.subplots(1, 1, figsize=[1.4, 1.2])

    d = data.loc[data['Experiment_Series'] == Series]

    sns.histplot(
        data=d.loc[d['is_HC']],
        x='num_hair_neighbors',
        hue='Experiment',
        # hue_order=[True, False],
        multiple='dodge',
        palette='tab10',
        stat='probability',
        common_norm=False,
        bins=[-0.4, 0.4, 0.6, 1.4, 1.6, 2.4, 2.6, 3.4, 3.6, 4.4]
    )

    ax.set_xlim([-0.5, 4.5])
    ax.set_xticks(range(5))
    ax.set_xlabel("Number of HCs around each HC")

    ax.set_ylim([0., 1])
    ax.set_yticks([0., 0.5, 1.])
    ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

    plt.savefig(f"{figure_path}/HC_contacts__{Series}.svg")


In [ ]:
fig, ax = plt.subplots(1, 1)

data = cells_experiments.loc[
    (
        (cells_experiments['Experiment_Series']=='MLCK') | 
        (cells_experiments['Experiment_Series']=='MLCK_16hrs')
    ) &
    (cells_experiments['is_HC'])
]

sns.violinplot(
    data=data,
    x='Experiment_Series',
    hue='is_Control', hue_order=[True, False],
    split=True,
    y='num_hair_neighbors',
)

ax.set_ylim([0, 3])


box_pairs=[
    (('MLCK', True), ('MLCK', False)),
    (('MLCK', True), ('MLCK_16hrs', True)),
    (('MLCK', False), ('MLCK_16hrs', False)),
    (('MLCK_16hrs', True), ('MLCK_16hrs', False))
]

annotator = Annotator(
    ax,
    box_pairs,
    data=data,
    x='Experiment_Series',
    hue='is_Control', hue_order=[True, False],
    split=True,
    y='num_hair_neighbors',
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75,
)
annotator.apply_and_annotate()

In [ ]:
fig, ax = plt.subplots(1, 1)

data = cells_experiments.loc[
    (
        (cells_experiments['Experiment_Series']=='ROCK') | 
        (cells_experiments['Experiment_Series']=='ROCK_16hrs')
    ) &
    (cells_experiments['is_HC'])
]

sns.violinplot(
    data=data,
    x='Experiment_Series',
    order=['ROCK', 'ROCK_16hrs'],
    hue='is_Control', hue_order=[True, False],
    split=True,
    y='num_hair_neighbors',
    # hue='is_Control',
    # hue_order=[True, False],
)

ax.set_ylim([0, 3])


box_pairs=[
    (('ROCK', True), ('ROCK', False)),
    (('ROCK', True), ('ROCK_16hrs', True)),
    (('ROCK', False), ('ROCK_16hrs', False)),
    (('ROCK_16hrs', True), ('ROCK_16hrs', False))
    # ((True, 'Vangl2Control'), (True, 'Vangl2KO')),
    # ((False, 'Vangl2Control'), (False, 'Vangl2KO')),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=data,
    x='Experiment_Series',
    order=['ROCK', 'ROCK_16hrs'],
    hue='is_Control', hue_order=[True, False],
    split=True,
    y='num_hair_neighbors',
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75,
)
annotator.apply_and_annotate()

## Area mapped

In [ ]:
def tissue_data_errorbar(
    *,
    x: str='HC_area',
    xlabel: str='Normalised HC area',
    y: str,
    ylabel: str=None,
    hue='Stage', hue_order=['E8', 'E10', 'E12', 'E14'],
    palette=palette_stages,
    color=None,
    style='P-D position',
    plot_control: bool=True, plot_errorbar=True,
    plot_treatment: bool=False, plot_errorbar_treatment=True,
    xlim=[0., 4.], ylim=None, xticks=None, yticks=None,
    figsize=[5.9/3, 5.9/4],
    savefig: str=None,
    mask=None,
    mask_treatment=None,
    tight_layout=True,
    data=None,
    **scatter_kwargs
):  
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    if tight_layout:
        fig.tight_layout()

    if mask is not None:
        data = tissue_data.loc[mask].copy()
    else:
        data = tissue_data.copy()


    if plot_control:
        sns.scatterplot(
            data=data,
            x=x,
            y=y,
            hue=hue, hue_order=hue_order,
            palette=palette,
            style=style,
            color=color,
            **scatter_kwargs
            # size='S-I position',
            # size_order=['Superior', 'Mid', 'Inferior']
        )
        if plot_errorbar:
            ax.errorbar(
                x=data[x],
                y=data[y],
                xerr=data["{}__std".format(x) ] if "{}__std".format(x) in data.columns else None,
                yerr=data["{}__std".format(y) ] if "{}__std".format(y) in data.columns else None,
                ls='',
                color='black', 
                linewidth=0.2, alpha=0.4,
                zorder=0
            )

    if plot_treatment:
        __data = tissue_data__experiments.copy()
        if mask_treatment is not None:
            __data = __data.loc[mask_treatment]
        __data = __data.loc[tissue_data__experiments['Experiment'] != 'Control']
        __data['__Experiment'] = __data['Experiment']
        __data['Experiment'] = __data['Experiment_Series']
        style_order=['Live', 'MLCK', 'MLCK_16hrs', 'Vangl2', 'ROCK', 'ROCK_16hrs', 'Atoh1', 'PTX']
        __data.Experiment = __data.Experiment.astype("category")
        __data.Experiment = __data.Experiment.cat.set_categories(style_order)
        sns.scatterplot(
            data=__data,
            x=x,
            y=y,
            hue='is_Control',
            hue_order=[True, False],
            palette=['blue', 'red'],
            style='Experiment',
        )
        if plot_errorbar_treatment:
            ax.errorbar(
                x=__data[x],
                y=__data[y],
                xerr=__data["{}__std".format(x) ] if "{}__std".format(x) in __data.columns else None,
                yerr=__data["{}__std".format(y) ] if "{}__std".format(y) in __data.columns else None,
                ls='',
                color='black', 
                linewidth=0.2, alpha=0.4,
                zorder=0
            )

    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel if ylabel is not None else y)
    if xticks is not None:
        ax.set_xticks(xticks)
    if yticks is not None:
        ax.set_yticks(yticks)

    mpl.pyplot.legend(bbox_to_anchor=(1,1.025), fontsize='xx-small')


    if savefig is not None:
        plt.savefig(f"{figure_path}/area_mapped__{savefig}.svg")
    return fig, ax


In [ ]:
tissue_data_errorbar(
    y='coordination_number', ylabel='HC neighbour no.',
    ylim=[3, 11],
    savefig='HC_neighbours',
)

In [ ]:
import scipy.optimize


fig, ax = tissue_data_errorbar(
    x='relative_area', xlabel='relative HC / SC area',
    y='coordination_number', ylabel='HC neighbour number',
    xlim=[0., 20],
    ylim=[3, 15],
    savefig='relative_area__HC_neighbours',
)

def num_SC(a, *args):
    # aligning with the long axis
    return math.pi * (a**0.5 + np.exp(-2*args[0])) * np.exp(-2*args[0])

popt_E12, pcov_E12 = scipy.optimize.curve_fit(
    num_SC,
    tissue_data.loc[tissue_data['stage']=='E12']['relative_area'],
    tissue_data.loc[tissue_data['stage']=='E12']['coordination_number'],
    p0=[0.],
    bounds=(-1, 1)
)
popt_E14, pcov_E14 = scipy.optimize.curve_fit(
    num_SC,
    tissue_data.loc[tissue_data['stage']=='E14']['relative_area'],
    tissue_data.loc[tissue_data['stage']=='E14']['coordination_number'],
    p0=[0.],
    bounds=(-1, 1)
)

print(popt_E12, np.sqrt(np.diag(pcov_E12)))
print(popt_E14, np.sqrt(np.diag(pcov_E14)))

a = np.linspace(0., 20., 101)
ax.plot(a, num_SC(a, 0.), color='black', ls='--', linewidth=linewidth)
ax.plot(a, num_SC(a, *popt_E12), color='brown', linewidth=linewidth)
ax.plot(a, num_SC(a, *popt_E14), color='orange', linewidth=linewidth)

plt.savefig(f"{figure_path}/area_mapped__relative_area__HC_neighbours.svg")


In [ ]:
fig, ax = tissue_data_errorbar(
    mask=tissue_data['stage']=='E14',
    x='relative_area', xlabel='relative HC / SC area',
    y='coordination_number', ylabel='HC neighbour number',
    xlim=[0., 20],
    ylim=[3, 12],
    savefig='relative_area__HC_neighbours',
)

ax.set_yticks([3, 6, 9, 12])
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(1))

a = np.linspace(0., 20., 101)
ax.plot(a, num_SC(a, 0.), color='black', ls='--', linewidth=linewidth)
ax.plot(a, num_SC(a, *popt_E14), color='orange', linewidth=linewidth)

plt.savefig(f"{figure_path}/area_mapped__relative_area__HC_neighbours__E14.svg")

In [ ]:
tissue_data_errorbar(
    x='relative_area', xlabel='relative HC / SC area',
    y='elongation_SC', ylabel='SC elongation |q|',
    xlim=[0,20], ylim=[0., 0.6],
    figsize=[1.5, 1.5], tight_layout=False,
    savefig='elongation_SC',
)
tissue_data_errorbar(
    mask=(tissue_data['stage']=='E14'),
    x='relative_area', xlabel='relative HC / SC area',
    y='elongation_SC', ylabel='SC elongation |q|',
    xlim=[0,20], ylim=[0., 0.6],
    figsize=[1.5, 1.5], tight_layout=False,
    savefig='elongation_SC__E14',
)
tissue_data_errorbar(
    x='relative_area', xlabel='relative HC / SC area',
    y='elongation_HC', ylabel='SC elongation |q|',
    xlim=[0,20], ylim=[0., 0.6],
    figsize=[1.5, 1.5], tight_layout=False,
    savefig='elongation_HC',
)

In [ ]:
tissue_data_errorbar(
    x='relative_area', xlabel='relative HC / SC area',
    y='SH_contacts', ylabel='SC-HC contacts',
    xlim=[0,20], ylim=[0., 4],    
    figsize=[1.5, 1.5], tight_layout=False,
    savefig='SH_contacts',
)
tissue_data_errorbar(
    mask=(tissue_data['stage']=='E14'),
    x='relative_area', xlabel='relative HC / SC area',
    y='SH_contacts', ylabel='SC-HC contacts',
    xlim=[0,20], ylim=[0., 4],
    figsize=[1.5, 1.5], tight_layout=False,
    savefig='SH_contacts__E14',
)

In [ ]:
tissue_data_errorbar(
    y='subordination_number', ylabel='SC neighbour no.',
    x='SC_area', xlabel='Normalised SC area',
    xlim=[0,1.5], ylim=[3, 8],
    savefig='SC_neighbours__SC_area',
)

In [ ]:

np.polyfit(tissue_data['HC_area'], tissue_data['subordination_number'], 1)

m_HC = np.polyfit(tissue_data['HC_area'], tissue_data['coordination_number'], 1)[0]
m_SC = np.polyfit(tissue_data['HC_area'], tissue_data['subordination_number'], 1)[0]
def correct_neighbors(nbs, area, is_HC):
    if is_HC:
        m = m_HC
    else:
        m = m_SC
    return nbs  + m * (1. - area)

tissue_data['coordination_number__corr'] = correct_neighbors(tissue_data['coordination_number'], tissue_data['HC_area'], True)
tissue_data['subordination_number__corr'] = correct_neighbors(tissue_data['subordination_number'], tissue_data['HC_area'], False)
tissue_data['delta_area'] = tissue_data['subordination_number__corr'] - tissue_data['coordination_number__corr']

In [ ]:
tissue_data_errorbar(
    y='coordination_number__corr', ylabel='HC neighbour no.',
    ylim=[3, 11],
)

In [ ]:
tissue_data_errorbar(
    y='delta_area', ylabel='HC neighbour no.',
    # ylim=[3, 11],
)

In [ ]:
tissue_data_errorbar(
    y='subordination_number__corr', ylabel='HC neighbour no.',
    ylim=[3, 11],
    savefig='total_neighbours',
)

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=tissue_data, x='HC_area', y='coordination_number')
ax.plot(np.linspace(0, 4, 101), np.linspace(0, 4, 101) * 1.514 + 3.865)

In [ ]:
fig, ax = tissue_data_errorbar(
    y='coordination_number', ylabel='HC neighbour number',
    ylim=[3, 11],
    plot_treatment=True,
    savefig=None,
    tight_layout=True
    # mask_treatment=mask_treatment
)

sns.regplot(tissue_data, x='HC_area', y='coordination_number', scatter=False,
            x_ci='sd', ci=99, line_kws=dict(lw=3*linewidth, color='black'))

ax.set_xlabel('norm. HC area')
ax.set_ylabel('HC neighbour number')

plt.savefig(f"{figure_path}/area_mapped__HC_neighbours__w_treatment.svg")


In [ ]:
mask_treatment = tissue_data__experiments['Experiment'] == 'Vangl2KO'
mask_treatment = np.where(
    tissue_data__experiments['Experiment'] == 'Vangl2Control',
    True,
    mask_treatment
)
print(np.unique(tissue_data__experiments.loc[mask_treatment]['filename']))
tissue_data_errorbar(
    y='coordination_number', ylabel='HC neighbour no.',
    ylim=[3, 11],
    plot_treatment=True,
    savefig='HC_neighbours__Vangl2KO',
    mask_treatment=mask_treatment
)

In [ ]:
mask_treatment = tissue_data__experiments['Experiment'] == 'MLCKdmso-4hrs'
mask_treatment = np.where(
    tissue_data__experiments['Experiment'] == 'MLCKdmso-16hrs',
    True,
    mask_treatment
)
mask_treatment = np.where(
    tissue_data__experiments['Experiment'] == 'MLCKml7-4hrs',
    True,
    mask_treatment
)
mask_treatment = np.where(
    tissue_data__experiments['Experiment'] == 'MLCKml7-16hrs',
    True,
    mask_treatment
)
print(np.unique(tissue_data__experiments.loc[mask_treatment]['filename']))
tissue_data_errorbar(
    y='coordination_number', ylabel='HC neighbour no.',
    ylim=[3, 11],
    plot_treatment=True,
    savefig='HC_neighbours__MLCK',
    mask_treatment=mask_treatment
)

In [ ]:
tissue_data_errorbar(
    y='subordination_number', ylabel='SC neighbour number',
    ylim=[3, 11],
    figsize=[2.75, 5.9/3],
    savefig='SC_neighbourhood',
)

In [ ]:
tissue_data_errorbar(
    y='subordination_number', ylabel='SC neighbour no.',
    ylim=[3, 9],
    plot_treatment=True,
    savefig='SC_neighbourhood__w_treatment',
)

In [ ]:
tissue_data_errorbar(
    y='neighbor_number', ylabel='Neighbour no.',
    ylim=[4, 8],
    savefig='topology',
)

In [ ]:
print(HC)

In [ ]:
fig, ax = plt.subplots(1, 1)

sns.regplot(data=HC, x='normalized_area_cells', y='num_neighbors',
            scatter=False, color='red')
sns.kdeplot(data=HC, x='normalized_area_cells', y='num_neighbors', alpha=0.3,
            color='orange')
sns.scatterplot(data=HC, x='normalized_area_cells', y='num_neighbors',
                style='position',
                style_order=position_ordering,
                hue='stage',
                hue_order=stage_ordering,
                palette=palette_stages,
                alpha=0.4,
                size=1,
                legend=False)

sns.lineplot(data=HC, x='HC_normalized_area', y='num_neighbors',
            **lineplot_position_stage_kwargs)

# ax.legend(bbox_to_anchor=(1.01, 1.05))

ax.set_xlim([0., 6])
ax.legend(fontsize='x-small')

plt.savefig(f"{figure_path}/num_neighbors__regplo.svg")

In [ ]:
tissue_data_errorbar(
    y='area__no_norm', ylabel='Cell area [$\mu m^2$]',
    ylim=[0, None],
    savefig='tissue_size',
)

In [ ]:
fig, ax = plt.subplots(1, 1)

sns.lineplot(data=HC, x='HC_normalized_area', y='shape_index',
             **lineplot_position_stage_kwargs)
_lineplot_position_stage_kwargs = lineplot_position_stage_kwargs.copy()
_lineplot_position_stage_kwargs['palette'] = 'cool'
sns.lineplot(data=SC, x='HC_normalized_area', y='shape_index', 
             **_lineplot_position_stage_kwargs)
ax.set_ylabel("Shape index")

ax.legend(bbox_to_anchor=(1.01, 1.05))

In [ ]:
tissue_data_errorbar(
    y='hexatic_order', ylabel='Hexatic order [$\psi_6$]',
    ylim=[0, 1],
    savefig='hexatic_order',
)

In [ ]:
tissue_data_errorbar(
    y='hexatic_order_corrected', ylabel='cor. Hexatic order [$\psi_6$]',
    ylim=[0, 1],
    savefig='hexatic_order_corrected',
)

In [ ]:
for p in range(3, 10):
    tissue_data_errorbar(
        y=f'{p}-atic_order', ylabel=f'{p}-atic order [$\psi_{p}$]',
        ylim=[0, 1],
        savefig=f'{p}-atic_order',
    )
    tissue_data_errorbar(
        y=f'{p}-atic_order_corrected', ylabel=f'{p}-atic order [$\psi_{p}^\star$]',
        ylim=[0, 1],
        savefig=f'{p}-atic_order_corrected',
    )

In [ ]:
tissue_data_errorbar(
    y='hexatic_order_corrected', ylabel='cor. Hexatic order [$\psi_6$]',
    ylim=[0, 1],
    plot_treatment=True,
    savefig='hexatic_order_corrected__w_treatment',
)

In [ ]:
fig, ax = tissue_data_errorbar(
    mask=((tissue_data['stage']=='E12') | (tissue_data['stage']=='E14')),
    x='relative_area', xlabel='relative HC / SC area',
    y='HC_density', ylabel='HC fraction',
    hue_order=['E8', 'E10', 'E12', 'E14'],
    xlim=[0, 20],
    ylim=[0, 0.4],
    figsize=[1.5, 1.5], tight_layout=False,
    savefig='relative_area__HC_density__E12_E14',
)

a = np.linspace(0., 20., 101)
ax.plot(a, 1. / (1. + num_SC(a, *popt_E12)/2.),
        color='brown', lw=linewidth)
ax.plot(a, 1. / (1. + num_SC(a, *popt_E14)/2.),
        color='orange', lw=linewidth)
ax.plot(a, 1. / (1. + num_SC(a, 0.)/2.),
        color='black', ls='--', lw=linewidth)
plt.savefig(f"{figure_path}/area_mapped__relative_area__HC_density__E12_E14.svg")


fig, ax = tissue_data_errorbar(
    mask=(tissue_data['stage']=='E14'),
    x='relative_area', xlabel='relative HC / SC area',
    y='HC_density', ylabel='HC fraction',
    hue_order=['E8', 'E10', 'E12', 'E14'],
    xlim=[0, 20],
    ylim=[0, 0.4],
    figsize=[1.5, 1.5], tight_layout=False,
    savefig='relative_area__HC_density__E14',
)

a = np.linspace(0., 20., 101)
ax.plot(a, 1. / (1. + num_SC(a, *popt_E14)/2.),
        color='orange', lw=linewidth)
ax.plot(a, 1. / (1. + num_SC(a, 0.)/2.),
        color='black', ls='--', lw=linewidth)
plt.savefig(f"{figure_path}/area_mapped__relative_area__HC_density__E14.svg")

In [ ]:
for pos in tissue_data['position'].unique():
    fig, ax = tissue_data_errorbar(
        mask=((tissue_data['position']==pos)),
        x='relative_area', xlabel='relative HC / SC area',
        y='HC_density', ylabel='HC fraction',
        hue_order=['E8', 'E10', 'E12', 'E14'],
        xlim=[0, 20],
        ylim=[0, 0.4],
        figsize=[1.5, 1.5], tight_layout=False,
        savefig=f'relative_area__HC_density__{pos}',
    )    
fig, ax = tissue_data_errorbar(
    mask=((tissue_data['position']=='25I') | (tissue_data['position']=='25S')),
    x='relative_area', xlabel='relative HC / SC area',
    y='HC_density', ylabel='HC fraction',
    hue_order=['E8', 'E10', 'E12', 'E14'],
    style=None,
    xlim=[0, 20],
    ylim=[0, 0.4],
    figsize=[1.5, 1.5], tight_layout=False,
    savefig=f'relative_area__HC_density__25IS',
)
fig, ax = tissue_data_errorbar(
    mask=((tissue_data['position']=='75I') | (tissue_data['position']=='75S')),
    x='relative_area', xlabel='relative HC / SC area',
    y='HC_density', ylabel='HC fraction',
    hue_order=['E8', 'E10', 'E12', 'E14'],
    style='P-D position',
    markers='s',
    xlim=[0, 20],
    ylim=[0, 0.4],
    figsize=[1.5, 1.5], tight_layout=False,
    savefig=f'relative_area__HC_density__75IS',
)

# a = np.linspace(0., 20., 101)
# ax.plot(a, 1. / (1. + num_SC(r_SC(a), *popt_E12)/2.),
#         color='brown', lw=linewidth)
# ax.plot(a, 1. / (1. + num_SC(r_SC(a), *popt_E14)/2.),
#         color='orange', lw=linewidth)
# ax.plot(a, 1. / (1. + num_SC(r_SC(a), 1.)/2.),
#         color='black', ls='--', lw=linewidth)
# plt.savefig(f"{figure_path}/area_mapped__relative_area__HC_density__25S.svg")

In [ ]:
tissue_data_errorbar(
    y='HC_density', ylabel='HC number density',
    ylim=[0, 0.4],
    plot_treatment=True,
    savefig='HC_density__w_treatment',
)

In [ ]:
tissue_data_errorbar(
    y='HS_bonds_normalised_length', ylabel='HC norm. bond length',
    # ylim=[0, 0.4],
    figsize=[2, 2*2/3], tight_layout=False,
    savefig='bonds_normalised_length__HS',
)

tissue_data_errorbar(
    y='SS_bonds_normalised_length', ylabel='SC-SC norm. bond length',
    # ylim=[0, 0.4],
    figsize=[2, 2*2/3], tight_layout=False,
    savefig='bonds_normalised_length__SS',
)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=[15, 6], sharey=True, sharex=True)
sns.pointplot(
    data=tissue_data,
    x='stage',
    y='HS_bonds_normalised_length',
    hue='position', hue_order=['0', '25I', '50I', '75I', '100'],
    palette='Reds',
    ax=axes[0]
)

sns.pointplot(
    data=tissue_data,
    x='stage',
    y='SS_bonds_normalised_length',
    hue='position', hue_order=['0', '25I', '50I', '75I', '100'],
    palette='Reds',
    ax=axes[1]
)

sns.pointplot(
    data=tissue_data,
    x='stage',
    y='bonds_normalised_length',
    hue='position', hue_order=['0', '25I', '50I', '75I', '100'],
    palette='Reds',
    ax=axes[2]
)

In [ ]:
fig, ax = plt.subplots(1, 1)

data = cells.groupby(by=['stage', 'position', 'SI_position', 'PD_position', 'file_id'])
data = data.mean()[['HC_normalized_area', 'is_HC']]
data = data.reset_index()

data = data.loc[data['stage'] != 'E13']
data['SI_position_long'] = data['SI_position'].where(data['SI_position'] != 'S', 'Superior')
data['SI_position_long'] = data['SI_position_long'].where(data['SI_position'] != 'None', 'Mid')
data['SI_position_long'] = data['SI_position_long'].where(data['SI_position'] != 'I', 'Inferior')

data['Stage'] = data['stage']
data['S-I position'] = data['SI_position_long']
data['P-D position'] = data['PD_position']

sns.scatterplot(
  data=data,
  x='HC_normalized_area', y='is_HC',
  hue='Stage',
  hue_order=stage_ordering,
  palette=palette_stages,
  style='P-D position',
  size='S-I position',
  size_order=['Superior', 'Mid', 'Inferior'],
)

ax.set_xlim([0., 6])
ax.set_ylim([0., 0.4])
ax.set_xlabel('Normalised HC area')
ax.set_ylabel('HC density')

data = cells.groupby(by='file_id')
_area = data['normalized_area_cells'].mean()
_area_std = data['normalized_area_cells'].std()

print(data.mean())

ax.errorbar(
  x=HC.groupby(by='file_id')['normalized_area_cells'].mean(),
  y=data['is_HC'].mean(),
  xerr=HC.groupby(by='file_id')['normalized_area_cells'].std(),
  yerr=data['is_HC'].count()**0.5/data['is_HC'].count(),
  ls='', color='black', 
  linewidth=0.2, alpha=0.5,
  zorder=0
)

plt.legend(prop={'size': 'small'})

In [ ]:
tissue_data_errorbar(
    y='polarity__variance', ylabel='polarity variance',
    # ylim=[0, 0.4],
    savefig='polarity_variance',
)

In [ ]:
tissue_data_errorbar(
    y='polarity__variance', ylabel='polarity variance',
    ylim=[0, None],
    plot_treatment=True,
    savefig='polarity_variance__w_treatment',
)

#### Hexatic order mapped

In [ ]:
tissue_data_errorbar(
    x='hexatic_order', xlabel='Hexatic order [$\psi_6$]',
    y='polarity__variance', ylabel='Polarity variance',
    xlim=[0, 1.],
    ylim=[0, None],
    figsize=[2, 1.5],
    # plot_treatment=True,
    savefig='polarity_variance__vs_hexatic_order__no_correction',
)

In [ ]:
tissue_data_errorbar(
    x='hexatic_order_corrected', xlabel='Hexatic order [$\psi_6$]',
    y='polarity_corrected__variance', ylabel='Polarity variance',
    xlim=[0, 1.],
    ylim=[0, 1.],
    figsize=[3, 1.6],
    # plot_treatment=True,
    savefig='polarity_variance__vs_hexatic_order',
)

In [ ]:
print(tissue_data.loc[tissue_data['stage']=='E10'].to_string())

In [ ]:
tissue_data_errorbar(
    x='hexatic_order_corrected', xlabel='Hexatic order [$\psi_6$]',
    y='polarity_corrected__stddev_angles', ylabel='Variation of polarity\n[c.s.d. in '+r'$\degree$]',
    style='aligned_with_PD_axis',
    xlim=[0, 0.8],
    ylim=[0, 140],
    xticks=[0., 0.4, 0.8],
    yticks=[0, 30, 60, 90, 120],
    figsize=[2.6, 2],
    plot_errorbar=False,
    savefig='polarity_stddev__vs_hexatic_order'
)

In [ ]:
mask = (tissue_data['position'] == '25I') | (tissue_data['position'] == '75I')
tissue_data_errorbar(
    x='hexatic_order_corrected', xlabel='Hexatic order [$\psi_6$]',
    y='polarity_corrected__variance', ylabel='Polarity variance',
    xlim=[0, 1.],
    ylim=[0, 1.],
    figsize=[3, 1.8],
    mask=mask,
    savefig='polarity_variance__vs_hexatic_order__25I_75I',
)

In [ ]:
for treatment in tissue_data__experiments['Experiment_Series'].unique():
    tissue_data_errorbar(
        x='hexatic_order_corrected', xlabel='Hexatic order [$\psi_6$]',
        y='polarity_corrected__stddev_angles', ylabel='Variation of polarity\n[c.s.d. in '+r'$\degree$]',
        style=None,
        xlim=[0, 1.],
        ylim=[0, 140],
        yticks=[0, 30, 60, 90, 120, 140],
        figsize=[3, 1.6],
        plot_errorbar=False,
        plot_treatment=True, plot_errorbar_treatment=False,
        mask_treatment=(tissue_data__experiments['Experiment_Series']==treatment),
        savefig='polarity_stddev__vs_hexatic_order__{}'.format(treatment),
    )

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[2.6, 2])
fig.tight_layout()

sns.regplot(
    data=tissue_data,
    x='hexatic_order_corrected',
    y='polarity_corrected__stddev_angles',
    # x_ci='sd', ci=None,
    color='black', scatter=False,
    line_kws={'ls': 'dashed', 'lw': linewidth},
    label='BP explants'
    # size='S-I position',
    # size_order=['Superior', 'Mid', 'Inferior']
)

data = tissue_data__experiments.copy()
data['__Experiment'] = data['Experiment']
data['Experiment'] = data['Experiment_Series']
data = data.loc[
    (data['Experiment']=='MLCK') |
    # (data['Experiment']=='MLCK_16hrs') |
    (data['Experiment']=='ROCK') |
    # (data['Experiment']=='ROCK_16hrs')
    # (data['Experiment']=='PTX') |
    (data['Experiment']=='Atoh1')
]
sns.scatterplot(
    data=data,
    x='hexatic_order_corrected',
    y='polarity_corrected__stddev_angles',
    style='is_Control',
    style_order=[True, False],
    # palette=['blue', 'red'],
    hue='Experiment',
)

order_limit = tissue_data.loc[
    (tissue_data['stage']=='E12') | (tissue_data['stage']=='E14'), 
    'hexatic_order_corrected'].min()
polarity_limit = tissue_data.loc[
    (tissue_data['stage']=='E12') | (tissue_data['stage']=='E14'), 
    'polarity_corrected__stddev_angles'].max()
ax.vlines(
    x=[order_limit],
    ymin=0, ymax=140,
    color='gray', alpha=0.7,
    lw=linewidth,
    zorder=-np.inf
)
ax.hlines(
    y=[polarity_limit],
    xmin=0, xmax=1,
    color='gray', alpha=0.7,
    lw=linewidth,
    zorder=-np.inf
)

# Create a Rectangle patch
rect = mpl.patches.Rectangle(
    (0, polarity_limit), 
    order_limit, 150,
    linewidth=0., edgecolor='none', 
    facecolor='red', alpha=0.1, 
    zorder=-np.inf
)
ax.add_patch(rect)
rect = mpl.patches.Rectangle(
    (order_limit, 0), 
    1., polarity_limit,
    linewidth=0., edgecolor='none', 
    facecolor='blue', alpha=0.1, 
    zorder=-np.inf
)
ax.add_patch(rect)

ax.set_xlim([0., 0.8])
ax.set_xticks([0., 0.4, 0.8])
ax.set_ylim([0., 140])
ax.set_yticks(range(0, 140, 30))

ax.set_xlabel('Hexatic order [$\psi_6$]')
ax.set_ylabel('Variation of polarity\n[c.s.d. in '+r'$\degree$]')

mpl.pyplot.legend(bbox_to_anchor=(1,1.025), fontsize='x-small')

plt.savefig(f"{figure_path}/area_mapped__polarity_stddev_vs_hexatic_order__inhibitions.svg")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[2.6, 2])
fig.tight_layout()

data = tissue_data__experiments.copy()
data['__Experiment'] = data['Experiment']
data['Experiment'] = data['Experiment_Series']
data = data.loc[
    (data['Experiment']=='MLCK') |
    (data['Experiment']=='ROCK') |
    (data['Experiment']=='Atoh1')
]
sns.scatterplot(
    data=data,
    x='hexatic_order_corrected',
    y='polarity_corrected__stddev_angles',
    style='is_Control',
    style_order=[True, False],
    hue='is_Control',
    hue_order=[True, False],
    palette=['blue', 'red'],
)

order_limit = tissue_data.loc[
    (tissue_data['stage']=='E12') | (tissue_data['stage']=='E14'), 
    'hexatic_order_corrected'].min()
polarity_limit = tissue_data.loc[
    (tissue_data['stage']=='E12') | (tissue_data['stage']=='E14'), 
    'polarity_corrected__stddev_angles'].max()
ax.vlines(
    x=[order_limit],
    ymin=0, ymax=140,
    color='gray', alpha=0.7,
    lw=linewidth,
    zorder=-np.inf
)
ax.hlines(
    y=[polarity_limit],
    xmin=0, xmax=1,
    color='gray', alpha=0.7,
    lw=linewidth,
    zorder=-np.inf
)

# Create a Rectangle patch
rect = mpl.patches.Rectangle(
    (0, polarity_limit), 
    order_limit, 150,
    linewidth=0., edgecolor='none', 
    facecolor='red', alpha=0.1, 
    zorder=-np.inf
)
ax.add_patch(rect)
rect = mpl.patches.Rectangle(
    (order_limit, 0), 
    1., polarity_limit,
    linewidth=0., edgecolor='none', 
    facecolor='blue', alpha=0.1, 
    zorder=-np.inf
)
ax.add_patch(rect)

ax.set_xlim([0., 0.8])
ax.set_xticks([0., 0.4, 0.8])
ax.set_ylim([0., 140])
ax.set_yticks(range(0, 140, 30))

ax.set_xlabel('Hexatic order [$\psi_6$]')
ax.set_ylabel('Variation of polarity\n[c.s.d. in '+r'$\degree$]')

mpl.pyplot.legend(bbox_to_anchor=(1,1.025), fontsize='x-small')

plt.savefig(f"{figure_path}/area_mapped__polarity_stddev_vs_hexatic_order__inhibitions__.svg")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[2.2, 1.6])

sns.regplot(
    data=tissue_data,
    x='hexatic_order_corrected',
    y='polarity_corrected__stddev_angles',
    # x_ci='sd', ci=None,
    color='black', scatter=False,
    line_kws={'ls': 'dashed', 'lw': linewidth},
    label='BP explants'
    # size='S-I position',
    # size_order=['Superior', 'Mid', 'Inferior']
)

data = tissue_data__experiments.copy()
data['__Experiment'] = data['Experiment']
data['Experiment'] = data['Experiment_Series']
data = data.loc[
    (data['Experiment']=='MLCK_16hrs') |
    (data['Experiment']=='ROCK_16hrs')
]
sns.scatterplot(
    data=data,
    x='hexatic_order_corrected',
    y='polarity_corrected__stddev_angles',
    style='is_Control',
    style_order=[True, False],
    hue='Experiment',
    palette=['tab:orange', 'tab:green'],
)

order_limit = tissue_data.loc[
    (tissue_data['stage']=='E12') | (tissue_data['stage']=='E14'), 
    'hexatic_order_corrected'].min()
polarity_limit = tissue_data.loc[
    (tissue_data['stage']=='E12') | (tissue_data['stage']=='E14'), 
    'polarity_corrected__stddev_angles'].max()
ax.vlines(
    x=[order_limit],
    ymin=0, ymax=140,
    color='gray', alpha=0.7,
    lw=linewidth,
    zorder=-np.inf
)
ax.hlines(
    y=[polarity_limit],
    xmin=0, xmax=1,
    color='gray', alpha=0.7,
    lw=linewidth,
    zorder=-np.inf
)

# Create a Rectangle patch
rect = mpl.patches.Rectangle(
    (0, polarity_limit), 
    order_limit, 150,
    linewidth=0., edgecolor='none', 
    facecolor='red', alpha=0.1, 
    zorder=-np.inf
)
ax.add_patch(rect)
rect = mpl.patches.Rectangle(
    (order_limit, 0), 
    1., polarity_limit,
    linewidth=0., edgecolor='none', 
    facecolor='blue', alpha=0.1, 
    zorder=-np.inf
)
ax.add_patch(rect)

ax.set_xlim([0., 0.8])
ax.set_ylim([0., 140])
ax.set_yticks(range(0, 140, 30))

ax.set_xlabel('Hexatic order [$\psi_6$]')
ax.set_ylabel('Variation of polarity\n[c.s.d. in '+r'$\degree$]')

mpl.pyplot.legend(bbox_to_anchor=(1,1.025), fontsize='x-small')

plt.savefig(f"{figure_path}/area_mapped__polarity_stddev_vs_hexatic_order_recovery.svg")

In [ ]:
import sklearn
import sklearn.cluster

X = np.asarray(data[['hexatic_order_corrected', 'polarity_corrected__stddev_angles']])
print(X)
kmeans = sklearn.cluster.KMeans(n_clusters=4).fit(X)
print(kmeans.labels_)

fig, ax = plt.subplots(1, 1, figsize=[2.4, 1.6])

ax.scatter(X[:,0], X[:,1], c=kmeans.labels_)
ax.set_xlim([0, 1])
ax.set_ylim([0, 140])
ax.set_yticks(range(0, 140, 30))


ax.set_xlabel('Hexatic order [$\psi_6$]')
ax.set_ylabel('Variation of polarity\n[c.s.d. in '+r'$\degree$]')


## Lewis' Law

In [ ]:
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

try: 
    os.mkdir(f"{figure_path}/Lewis_law")
except OSError as error: 
    print(error)


for stage in ['E8-E12', 'E8', 'E10', 'E12', 'E14', 'all']:
    for type in ['all', 'HC', 'SC']:
        fig, ax = plt.subplots(1, 1, figsize=[5.9/3, 5.9/3])

        data = cells.loc[(cells['position'] == '25I')]

        if stage == 'E8' or stage == 'E10' or stage == 'E12' or stage == 'E14':
            data = data.loc[data['stage']==stage]
        if stage == 'E8-E12':
            data = data.loc[data['stage']!='E14']
        
        if type == 'HC':
            data = data[ data['is_HC']]
        if type == 'SC':
            data = data[~data['is_HC']]

        norm = data.groupby(by='file_id').apply(lambda grp: grp['area_cells'].mean())
        norm = norm.rename('A').reset_index()

        groubby_n = data.groupby(by=['file_id', 'stage', 'position', 'num_neighbors'])
        # A_n = groubby_n.apply(lambda grp, *, norm: print(norm.loc[norm['file_id']==grp['file_id'].iloc[0], 'A'].iloc[0]), norm=norm)
        A_n = groubby_n.apply(lambda grp, *, norm: grp['area_cells'].mean() / norm.loc[norm['file_id']==grp['file_id'].iloc[0], 'A'].iloc[0] if grp['area_cells'].count() >= 5 else np.nan, norm=norm)
        A_n = A_n.rename('A_n/A').reset_index()

        sns.scatterplot(
            data=A_n,
            x='num_neighbors',
            y='A_n/A',
            hue='stage',
            hue_order=stage_ordering,
            markers=['X'],
            style='position',
            palette=palette_stages,
        )
        sns.regplot(
            data=A_n,
            x='num_neighbors',
            y='A_n/A',
            scatter=False,
            x_ci='sd',
            label='Lin. Regression',
            line_kws={
                'color': 'red',
                'lw': linewidth
            }
        )

        _ns = np.array([i for i in range(3, 13)])
        ax.plot(
            _ns,
            (_ns - 2) / 4,
            label='Lewis Law',
            color='black',
            lw=linewidth
        )

        ax.legend(bbox_to_anchor=(1.01, 1.05))

        ax.set_xlabel('Polygon type (n)')
        ax.set_ylabel(r'$\frac{\overline{A_n}}{\overline{A}}$', rotation='horizontal')

        ax.set_xlim([2.5, 12.5])
        ax.set_xticks(range(4, 13, 2))
        ax.xaxis.set_minor_locator(MultipleLocator(1))

        if type != 'HC':
            ax.set_ylim([0,5])
            ax.set_yticks(range(0, 6))
            ax.yaxis.set_minor_locator(MultipleLocator(0.5))
        else:
            ax.set_ylim([0,3])
            ax.set_yticks(range(0, 4))
            ax.yaxis.set_minor_locator(MultipleLocator(0.5))
            

        plt.savefig(f"{figure_path}/Lewis_law/{stage}__{type}.svg")
        plt.close()

In [ ]:
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)


try: 
    os.mkdir(f"{figure_path}/Lewis_law")
except OSError as error: 
    print(error)


stage = 'E8-E12'
type = 'HC'

fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])


data = cells.loc[(cells['position'] == '25I')]
data = data.loc[data['stage']!='E14']
data = data[~data['is_HC']]

norm = data.groupby(by='file_id').apply(lambda grp: grp['area_cells'].mean())
norm = norm.rename('A').reset_index()

groubby_n = data.groupby(by=['file_id', 'stage', 'position', 'num_neighbors'])
A_n = groubby_n.apply(lambda grp, *, norm: grp['area_cells'].mean() / norm.loc[norm['file_id']==grp['file_id'].iloc[0], 'A'].iloc[0] if grp['area_cells'].count() >= 5 else np.nan, norm=norm)
A_n = A_n.rename('A_n/A').reset_index()

sns.scatterplot(
    data=A_n,
    x='num_neighbors',
    y='A_n/A',
    markers=['X'],
    color='gray',
    label = 'SCs & lin. reg.'
)
sns.regplot(
    data=A_n,
    x='num_neighbors',
    y='A_n/A',
    scatter=False,
    x_ci='sd',
    line_kws={
        'color': 'gray',
        'lw': 2*linewidth
    }
)


data = cells.loc[(cells['position'] == '25I')]
data = data.loc[data['stage']!='E14']
data = data[ data['is_HC']]

norm = data.groupby(by='file_id').apply(lambda grp: grp['area_cells'].mean())
norm = norm.rename('A').reset_index()

groubby_n = data.groupby(by=['file_id', 'stage', 'position', 'num_neighbors'])
A_n = groubby_n.apply(lambda grp, *, norm: grp['area_cells'].mean() / norm.loc[norm['file_id']==grp['file_id'].iloc[0], 'A'].iloc[0] if grp['area_cells'].count() >= 5 else np.nan, norm=norm)
A_n = A_n.rename('A_n/A').reset_index()

sns.scatterplot(
    data=A_n,
    x='num_neighbors',
    y='A_n/A',
    markers=['X'],
    color='red',
    label = 'HCs & lin. reg.'
)
sns.regplot(
    data=A_n,
    x='num_neighbors',
    y='A_n/A',
    scatter=False,
    x_ci='sd',
    # label='Lin. Reg. HCs',
    line_kws={
        'color': 'red',
        'lw': 2*linewidth
    }
)
    


_ns = np.array([i for i in range(3, 12)])
ax.plot(
    _ns,
    (_ns - 2) / 4,
    label='Lewis Law',
    color='black',
    lw=2*linewidth
)



ax.legend(loc='upper left')
# ax.legend(bbox_to_anchor=(1.01, 1.05))

ax.set_xlabel('Polygon type (n)')
ax.set_ylabel(r'$\frac{\overline{A_n}}{\overline{A}}$', rotation='horizontal')

ax.set_xlim([2.5, 11.5])
ax.set_xticks(range(4, 12, 2))
ax.xaxis.set_minor_locator(MultipleLocator(1))

ax.set_ylim([0,3.5])
ax.set_yticks(range(0, 4))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))

plt.savefig(f"{figure_path}/Lewis_law/{stage}__merge.svg")

In [ ]:
fig, ax = plt.subplots(1, 1)

data = cells.loc[(cells['stage'] == 'E10') & (cells['is_HC'])]

groubby_n = data.groupby(by=['file_id', 'stage', 'position', 'num_neighbors'])
# groubby_n.apply(lambda d: print(d[['frame_nb', 'num_neighbors', 'normalized_area_cells']]))
A_n = groubby_n.apply(lambda grp: grp['normalized_area_cells'].mean())
A_n = A_n.rename('A_n').reset_index()

sns.scatterplot(
    data=A_n,
    x='num_neighbors',
    y='A_n',
    hue='stage',
    hue_order=stage_ordering,
    style='position',
    style_order=position_ordering,
    palette=palette_stages,
)
sns.regplot(
    data=A_n,
    x='num_neighbors',
    y='A_n',
    scatter=False,
    label='lin. reg.'
)
sns.regplot(
    data=A_n,
    x='num_neighbors',
    y='A_n',
    scatter=False,
    order=2,
    label='quad. reg.'
)

ax.set_title("all cells")

_ns = np.array([i for i in range(2, 14)])
ax.plot(_ns, (_ns - 2) / 4, label='Lewis Law')

ax.plot(_ns, _ns / 6 * np.tan(math.pi / 6) / np.tan(math.pi / _ns), label='Quadr. Lewis Law')
ax.legend(bbox_to_anchor=(1.01, 1.05))


plt.savefig(f"{figure_path}/Lewis_law/E10_HC.svg")

In [ ]:
print(cells_experiments['Experiment'].unique())

In [ ]:
_HC = HC.copy()
_HC = _HC.loc[_HC['stage'] == 'E10']
_HC = _HC.loc[_HC['position'] == '25I']

_SC = SC.copy()
_SC = _SC.loc[_SC['stage'] == 'E10']
_SC = _SC.loc[_SC['position'] == '25I']

print("Control")
print("HC norm area: {:.2f}".format(_HC['normalized_area_cells'].mean()))
print("SC norm area: {:.2f}".format(_SC['normalized_area_cells'].mean()))
print("Coordination number: {:.1f} (H: {:.2f}, S: {:.1f})".format(
    _HC['num_neighbors'].mean(),
    _HC['num_hair_neighbors'].mean(),
    _HC['num_neighbors'].mean() - _HC['num_hair_neighbors'].mean()))
print("Subordination number: {:.1f} (H: {:.1f}, S: {:.1f})".format(
    _SC['num_neighbors'].mean(),
    _SC['num_hair_neighbors'].mean(),
    _SC['num_neighbors'].mean() - _SC['num_hair_neighbors'].mean()))

_hc = cells_experiments.loc[cells_experiments['is_HC']]
_hc = _hc.loc[_hc['Experiment'] == 'MLCKml7-4hrs']
_hc = _hc.loc[_hc['stage'] == 'E10']
_hc = _hc.loc[_hc['position'] == '25I']

_sc = cells_experiments.loc[~cells_experiments['is_HC']]
_sc = _sc.loc[_sc['Experiment'] == 'MLCKml7-4hrs']
_sc = _sc.loc[_sc['stage'] == 'E10']
_sc = _sc.loc[_sc['position'] == '25I']

print("\nML-7")
print("HC norm area: {:.2f}".format(_hc['normalized_area_cells'].mean()))
print("SC norm area: {:.2f}".format(_sc['normalized_area_cells'].mean()))
print("Coordination number: {:.1f} (H: {:.2f}, S: {:.1f})".format(
    _hc['num_neighbors'].mean(),
    _hc['num_hair_neighbors'].mean(),
    _hc['num_neighbors'].mean() - _hc['num_hair_neighbors'].mean()))
print("Subordination number: {:.1f} (H: {:.1f}, S: {:.1f})".format(
    _sc['num_neighbors'].mean(),
    _sc['num_hair_neighbors'].mean(),
    _sc['num_neighbors'].mean() - _sc['num_hair_neighbors'].mean()))

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, sharex=True)

sns.lineplot(data=SC.loc[SC['stage'] == 'E10'], x='num_neighbors', y='normalized_area_cells', hue='stage', style='position', ci='sd', palette='cool', ax=ax1, markers=True, dashes=False, ls='', err_style='bars')
sns.lineplot(data=HC.loc[HC['stage'] == 'E10'], x='num_neighbors', y='normalized_area_cells', hue='stage', style='position', ci='sd', palette=palette_stages, ax=ax2, markers=True, dashes=False, ls='', err_style='bars')
sns.lineplot(data=cells.loc[cells['stage'] == 'E10'], x='num_neighbors', y='normalized_area_cells', hue='stage', style='position', ci='sd', palette=palette_stages, ax=ax3, markers=True, dashes=False, ls='', err_style='bars')

 
ax1.legend(bbox_to_anchor=(2.8, 1.05))
ax2.legend(bbox_to_anchor=(2.2, 1.05))


plt.savefig(f"{figure_path}/A_n.svg")

## Hexatic order

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/5, 5.9/3])

points = sns.pointplot(
    data=cells,
    x='stage',
    y='hexatic_order_corrected',
    hue='position', hue_order=['25I', '75I'],
    # palette='Oranges',
    join=False,
    ci='sd',
    # capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
)
plt.setp(points.collections, sizes=[markersize])

results__hexatic_order = pd.DataFrame()
results__hexatic_order['hexatic_order'] = cells.groupby(by=['stage', 'position'])['hexatic_order_corrected'].mean()
results__hexatic_order['hexatic_order__std'] = cells.groupby(by=['stage', 'position'])['hexatic_order_corrected'].std()

print(results__hexatic_order.to_string())

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['25I', '75I'], title='')

ax.set_ylim([0, 1])

ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

ax.set_ylabel('Hexatic order $\psi_6$')
ax.set_yticks([0., 0.5, 1.])


plt.savefig(f"{figure_path}/hexatic_vs_stage__explants.svg")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['position']=='25I'],
    x='stage',
    y='hexatic_order_corrected',
    # hue='position', hue_order=['25I'],
    color='red',
    join=False,
    errorbar='sd',
    # capsize=.05,
    # dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

results__hexatic_order = pd.DataFrame()
results__hexatic_order['hexatic_order'] = cells.groupby(by=['stage', 'position'])['hexatic_order_corrected'].mean()
results__hexatic_order['hexatic_order__std'] = cells.groupby(by=['stage', 'position'])['hexatic_order_corrected'].std()

print(results__hexatic_order.to_string())

# leg_handles = ax.get_legend_handles_labels()[0]
# ax.legend(leg_handles, ['25I', '75I'], title='')

ax.set_ylim([0, 1])

ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

ax.set_ylabel('Hexatic order $\psi_6$')
ax.set_yticks([0., 0.5, 1.])


plt.savefig(f"{figure_path}/hexatic_vs_stage__25I.svg")

## Density mapped

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
fig.tight_layout()

points = sns.scatterplot(
    data=tissue_data,
    x='HC_density',
    # order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='HC_area',
    hue='stage',
    hue_order=['E8', 'E10', 'E12', 'E14'],
    palette='Reds',
    # join=False,
    # errorbar='sd',
    # capsize=.05,
    # dodge=True,
    # s=2,
    # errwidth=linewidth,
    markers='x',
)
plt.savefig(f"{figure_path}/density_mapped__HC_area.svg",format='svg')
    
for stage in ['E8', 'E10', 'E12', 'E14']:
    fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
    fig.tight_layout()

    points = sns.scatterplot(
        data=tissue_data.loc[tissue_data['stage'] == stage],
        x='HC_density',
        # order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
        y='HC_area',
        color='Red',
        # hue='is_HC',
        # hue_order=[False, True],
        # palette=['gray', 'red'],
        # join=False,
        # errorbar='sd',
        # capsize=.05,
        # dodge=True,
        # s=2,
        # errwidth=linewidth,
        marker='x',
    )
    plt.savefig(f"{figure_path}/density_mapped__HC_area__{stage}.svg",format='svg')

In [ ]:
for stage in ['E8', 'E10', 'E12', 'E14']:
    tissue_data_errorbar(
        x='HC_density', xlabel='HC fraction',
        y='HC_area', ylabel='Normalised HC area',
        hue=None,
        color='Red',
        xlim=[0.1, 0.4], ylim=[0., 5.],
        xticks=None, yticks=None,
        savefig=f"density_mapped__HC_area__{stage}.svg",
        mask=(tissue_data['stage']==stage),
    )

In [ ]:
for stage in ['E8', 'E10', 'E12', 'E14']:
    tissue_data_errorbar(
        x='HC_density', xlabel='HC fraction',
        y='relative_area', ylabel='rel. HC / SC area',
        hue=None,
        color='Red',
        xlim=[0.1, 0.4], ylim=[0., 20.],
        xticks=None, yticks=None,
        figsize=[2., 1.5],
        savefig=f"density_mapped__rel_area__{stage}.svg",
        mask=(tissue_data['stage']==stage),
    )

In [ ]:
simulation_data = pd.read_csv('/mnt/DataSSD/weninger/utopia_output/PCPTopology/241015-120453_job140732_2D_plus__increment_experiment__512_cells/eval/rho_mapped/area/HC_raw_data.csv')
print(simulation_data)
simulation_data = simulation_data.groupby(by='rho')['area'].mean()
print(simulation_data)

In [ ]:
exp_data = tissue_data.loc[(tissue_data['stage']=='E14') & (tissue_data['Experiment']=='Control')].copy().sort_values(by='HC_density')
exp_data['sim_area'] = np.interp(exp_data['HC_density'], simulation_data.index, simulation_data)
print(exp_data[['HC_density', 'HC_area', 'sim_area']])


In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize=[6, 6])
sns.scatterplot(
    data=exp_data, 
    x='HC_density',
    y='HC_area', ax=ax1
)
sns.scatterplot(
    data=exp_data,
    x='HC_density',
    y='sim_area', ax=ax1
)
sns.lineplot(
    data=simulation_data.reset_index(),
    x='rho',
    y='area', ax=ax1
)

sns.regplot(
    data=exp_data, 
    x='HC_area',
    y='sim_area', ax=ax2
)
r, p = scipy.stats.pearsonr(exp_data['HC_area'], exp_data['sim_area'])
ax2.annotate(text=f"Pearson r: {r:.2f}", xy=(1,3.5))
ax2.set_xlim(0, 4.5)
ax2.set_ylim(0, 4.5)
ax2.set_aspect(1)

plt.savefig(f"{figure_path}/density_mapped_area_E14__correllation.svg",format='svg')


In [ ]:
for stage in ['E8', 'E10', 'E12', 'E14']:
    tissue_data_errorbar(
        x='HC_density', xlabel='HC fraction',
        y='hexatic_order_corrected', ylabel='Hexatic order $\psi_6$',
        hue=None,
        color='Red',
        xlim=[0.1, 0.4], ylim=[0., 1.],
        xticks=None, yticks=np.linspace(0., 1., 6),
        savefig=f"density_mapped__hexatic_order__{stage}.svg",
        mask=(tissue_data['stage']==stage),
    )
    

In [ ]:
print(tissue_data.to_string())

In [ ]:
for stage in ['E8', 'E10', 'E12', 'E14']:
    fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
    fig.tight_layout()

    points = sns.scatterplot(
        data=tissue_data.loc[tissue_data['stage'] == stage],
        x='HC_density',
        # order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
        y='HC_area__no_norm',
        # hue='is_HC',
        # hue_order=[False, True],
        # palette=['gray', 'red'],
        # join=False,
        # errorbar='sd',
        # capsize=.05,
        # dodge=True,
        # s=2,
        # errwidth=linewidth,
        markers='x',
    )
    plt.savefig(f"{figure_path}/density_mapped__HC_area_no_norm__{stage}__no_norm.svg",format='svg')


## Spatial gradients

In [ ]:
for stage in ['E8', 'E10', 'E12', 'E14']:
    fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
    fig.tight_layout()

    points = sns.pointplot(
        data=cells.loc[cells['stage'] == stage],
        x='position',
        order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
        y='area',
        hue='is_HC',
        hue_order=[False, True],
        palette=['gray', 'red'],
        join=False,
        errorbar='sd',
        capsize=.05,
        dodge=True,
        # s=2,
        errwidth=linewidth,
        markers='x',
    )
    plt.setp(points.collections, sizes=[markersize], linewidths=[2*linewidth])

    ax.set_ylim([0, 90])
    ax.set_yticks([0, 30, 60, 90]) 
    ax.set_xlim([-0.5, 6.5])

    ax.set_ylabel('Cell area [$\mu m^2$]')

    ax.set_xlabel('')
    ax.set_xticklabels(['25I', '25S', '50I', '50S', '75I', '75S', '100'], fontsize=labelsize)

    leg_handles = ax.get_legend_handles_labels()[0]
    ax.legend(leg_handles, ['SC', 'HC'], title='')

    plt.savefig(f"{figure_path}/HC_area__{stage}__no_norm.svg",format='svg')

In [ ]:
for stage in ['E8', 'E10', 'E12', 'E14']:
    fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
    fig.tight_layout()

    points = sns.pointplot(
        data=tissue_data.loc[tissue_data['stage'] == stage],
        x='position',
        order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
        y='HC_area__no_norm',
        color='red',
        join=False,
        errorbar='sd',
        capsize=.05,
        dodge=True,
        # s=2,
        errwidth=linewidth,
        markers='x',
    )
    plt.setp(points.collections, sizes=[markersize], linewidths=[2*linewidth])

    points = sns.pointplot(
        data=tissue_data.loc[tissue_data['stage'] == stage],
        x='position',
        order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
        y='SC_area__no_norm',
        color='gray',
        join=False,
        errorbar='sd',
        capsize=.05,
        dodge=True,
        # s=2,
        errwidth=linewidth,
        markers='x',
    )
    plt.setp(points.collections, sizes=[markersize], linewidths=[2*linewidth])

    ax.set_xlim([-0.5, 6.5])
    ax.set_xlabel('')
    ax.set_xticklabels(['25I', '25S', '50I', '50S', '75I', '75S', '100'], fontsize=labelsize)

    ax.set_ylabel('Cell area [$\mu m^2$]')
    # if stage != 'E14':
    #     ax.set_ylim([0, 30])
    #     ax.set_yticks([0, 10, 20, 30])
    # else:
    ax.set_ylim([0, 90])
    ax.set_yticks([0, 30, 60, 90])

    leg_handles = ax.get_legend_handles_labels()[0]
    print(leg_handles)
    ax.legend(leg_handles, labels=['SC', 'HC'], title='')

    plt.savefig(f"{figure_path}/HC_area__{stage}__no_norm__tissue.svg",format='svg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
fig.tight_layout()

points = sns.pointplot(
    data=tissue_data,
    x='position',
    order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='SC_area__no_norm',
    hue='stage',
    color='gray',
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='x',
)
plt.setp(points.collections, sizes=[markersize], linewidths=[2*linewidth])

ax.set_xlim([-0.5, 6.5])
ax.set_xlabel('')
ax.set_xticklabels(['25I', '25S', '50I', '50S', '75I', '75S', '100'], fontsize=labelsize)

ax.set_ylabel('Cell area [$\mu m^2$]')
# if stage != 'E14':
#     ax.set_ylim([0, 30])
#     ax.set_yticks([0, 10, 20, 30])
# else:
ax.set_ylim([0, 20])
ax.set_yticks([0, 5, 10, 15, 20])

# leg_handles = ax.get_legend_handles_labels()[0]
# print(leg_handles)
# ax.legend(leg_handles, labels=['SC', 'HC'], title='')

plt.savefig(f"{figure_path}/SC_area__no_norm__tissue.svg",format='svg')

In [ ]:
for stage in ['E8', 'E10', 'E12', 'E14']:
    fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
    fig.tight_layout()

    points = sns.pointplot(
        data=cells.loc[cells['stage'] == stage],
        x='position',
        order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
        y='num_neighbors',
        hue='is_HC',
        hue_order=[False, True],
        palette=['gray', 'red'],
        join=False,
        errorbar='sd',
        capsize=.05,
        dodge=True,
        # s=2,
        errwidth=linewidth,
        markers='x',
    )
    plt.setp(points.collections, sizes=[markersize], linewidths=[2*linewidth])

    ax.set_ylim([3, 12])
    ax.hlines(y=[6], xmin=-0.5, xmax=6.5, color='black', ls=':', linewidth=linewidth)
    ax.set_yticks([3, 6, 9, 12])
    ax.set_xlim([-0.5, 6.5])

    ax.set_ylabel('Neighbour number')

    ax.set_xlabel('')
    ax.set_xticklabels(['25I', '25S', '50I', '50S', '75I', '75S', '100'], fontsize=labelsize)

    leg_handles = ax.get_legend_handles_labels()[0]
    ax.legend(leg_handles, ['SC', 'HC'], title='')

    plt.savefig(f"{figure_path}/Neighbours__{stage}.svg",format='svg')

In [ ]:
for stage in ['E8', 'E10', 'E12', 'E14']:
    fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
    fig.tight_layout()

    points = sns.pointplot(
        data=tissue_data.loc[tissue_data['stage'] == stage],
        x='position',
        order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
        y='coordination_number',
        color='red',
        join=False,
        errorbar='sd',
        capsize=.05,
        dodge=True,
        # s=2,
        errwidth=linewidth,
        markers='x',
    )
    plt.setp(points.collections, sizes=[markersize], linewidths=[2*linewidth])

    points = sns.pointplot(
        data=tissue_data.loc[tissue_data['stage'] == stage],
        x='position',
        order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
        y='subordination_number',
        color='gray',
        join=False,
        errorbar='sd',
        capsize=.05,
        dodge=True,
        # s=2,
        errwidth=linewidth,
        markers='x',
    )
    plt.setp(points.collections, sizes=[markersize], linewidths=[2*linewidth])

    ax.set_ylim([3, 12])
    ax.hlines(y=[6], xmin=-0.5, xmax=6.5, color='black', ls=':', linewidth=linewidth)
    ax.set_yticks([3, 6, 9, 12])
    ax.set_xlim([-0.5, 6.5])

    ax.set_ylabel('Neighbour number')

    ax.set_xlabel('')
    ax.set_xticklabels(['25I', '25S', '50I', '50S', '75I', '75S', '100'], fontsize=labelsize)

    leg_handles = ax.get_legend_handles_labels()[0]
    ax.legend(leg_handles, ['SC', 'HC'], title='')

    plt.savefig(f"{figure_path}/Neighbours__{stage}__tissue.svg",format='svg')

In [ ]:
for stage in ['E8', 'E10', 'E12', 'E14']:
    fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
    fig.tight_layout()

    points = sns.pointplot(
        data=cells.loc[cells['stage'] == stage],
        x='position',
        order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
        y='hexatic_order_corrected',
        hue='is_HC',
        hue_order=[True],
        palette=['red'],
        join=False,
        errorbar='sd',
        capsize=.05,
        dodge=True,
        # s=2,
        errwidth=linewidth,
        markers='x',
    )
    plt.setp(points.collections, sizes=[markersize], linewidths=[2*linewidth])

    ax.set_ylim([0, 1.])
    ax.set_yticks([0., 0.5, 1.])
    ax.set_xlim([-0.5, 6.5])

    ax.set_ylabel('Hexatic order $\psi_6$')

    ax.set_xlabel('')
    ax.set_xticklabels(
        ['25I', '25S', '50I', '50S', '75I', '75S', '100'],
        fontsize=labelsize
    )

    leg_handles = ax.get_legend_handles_labels()[0]
    ax.legend(leg_handles, ['HC'], title='')

    plt.savefig(f"{figure_path}/Hexatic_order__{stage}.svg",format='svg')

In [ ]:
for stage in ['E8', 'E10', 'E12', 'E14']:
    fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
    fig.tight_layout()

    points = sns.pointplot(
        data=tissue_data.loc[tissue_data['stage'] == stage],
        x='position',
        order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
        # y='6-atic_order_corrected',
        y='hexatic_order_corrected',
        color='red',
        join=False,
        errorbar='sd',
        capsize=.05,
        dodge=True,
        # s=2,
        errwidth=linewidth,
        markers='x',
    )
    plt.setp(points.collections, sizes=[markersize], linewidths=[2*linewidth])

    ax.set_ylim([0, 1.])
    ax.set_yticks([0., 0.5, 1.])
    ax.set_xlim([-0.5, 6.5])

    ax.set_ylabel('Hexatic order $\psi_6$')

    ax.set_xlabel('')
    ax.set_xticklabels(
        ['25I', '25S', '50I', '50S', '75I', '75S', '100'],
        fontsize=labelsize
    )

    leg_handles = ax.get_legend_handles_labels()[0]
    ax.legend(leg_handles, ['HC'], title='')

    plt.savefig(f"{figure_path}/Hexatic_order__{stage}__tissue.svg",format='svg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
fig.tight_layout()

points = sns.pointplot(
    data=tissue_data.loc[(tissue_data['stage'] == 'E8') | (tissue_data['stage'] == 'E14')],
    x='position',
    order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='hexatic_order_corrected',
    # y='6-atic_order_corrected',
    hue='stage', hue_order=['E14', 'E8'],
    color='red',
    palette=['red', 'black'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='x',
)
plt.setp(points.collections, sizes=[markersize], linewidths=[2*linewidth])


points = sns.stripplot(
    data=tissue_data.loc[(tissue_data['stage'] == 'E8') | (tissue_data['stage'] == 'E14')],
    x='position',
    order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='hexatic_order_corrected',
    # y='6-atic_order_corrected',
    hue='stage', hue_order=['E14', 'E8'],
    color='red',
    palette=['red', 'black'],
    alpha=0.8,
    size=1.5,
    # scale=5,
    # markers='.',
)

ax.set_ylim([0, 1.])
ax.set_yticks([0., 0.5, 1.])
ax.set_xlim([-0.5, 6.5])

ax.set_ylabel('Hexatic order $\psi_6$')

ax.set_xlabel('')
ax.set_xticklabels(
    ['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    fontsize=labelsize
)

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['E14', 'E8'], title='')

plt.savefig(f"{figure_path}/Hexatic_order__E8_E14__tissue.svg",format='svg')

box_pairs = [((pos, 'E14'), (pos, 'E8')) for pos in ['25I', '25S', '50I', '50S', '75I', '75S', '100']]
print(box_pairs)

annotator = Annotator(
    ax,
    box_pairs,
    data=tissue_data.loc[(tissue_data['stage'] == 'E8') | (tissue_data['stage'] == 'E14')],
    x='position',
    order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    hue='stage',  
    hue_order=['E14', 'E8'],
    y='hexatic_order_corrected',
    split=False,
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75,
    loc='outside',
    text_format='star'
)
annotator.apply_and_annotate()

plt.savefig(f"{figure_path}/Hexatic_order__E8_E14__tissue__stats.svg",format='svg')


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
fig.tight_layout()

points = sns.pointplot(
    data=tissue_data.loc[(tissue_data['stage'] == 'E10') | (tissue_data['stage'] == 'E14')],
    x='position',
    order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='hexatic_order_corrected',
    # y='6-atic_order_corrected',
    hue='stage', hue_order=['E14', 'E10'],
    color='red',
    palette=['red', 'black'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='x',
)
plt.setp(points.collections, sizes=[markersize], linewidths=[2*linewidth])


points = sns.stripplot(
    data=tissue_data.loc[(tissue_data['stage'] == 'E10') | (tissue_data['stage'] == 'E14')],
    x='position',
    order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='hexatic_order_corrected',
    # y='6-atic_order_corrected',
    hue='stage', hue_order=['E14', 'E10'],
    color='red',
    palette=['red', 'black'],
    alpha=0.8,
    size=1.5,
    # scale=5,
    # markers='.',
)
# plt.setp(points.collections, sizes=[2*markersize], linewidths=[2*linewidth])
# plt.setp(points.collections, sizes=[.25*markersize])


ax.set_ylim([0, 1.])
ax.set_yticks([0., 0.5, 1.])
ax.set_xlim([-0.5, 6.5])

ax.set_ylabel('Hexatic order $\psi_6$')

ax.set_xlabel('')
ax.set_xticklabels(
    ['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    fontsize=labelsize
)

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['E14', 'E10'], title='')

plt.savefig(f"{figure_path}/Hexatic_order__E10_E14__tissue.svg",format='svg')

box_pairs = [((pos, 'E14'), (pos, 'E10')) for pos in ['25I', '25S', '50I', '50S', '75I', '75S', '100']]
print(box_pairs)

annotator = Annotator(
    ax,
    box_pairs,
    data=tissue_data.loc[(tissue_data['stage'] == 'E10') | (tissue_data['stage'] == 'E14')],
    x='position',
    order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    hue='stage',  
    hue_order=['E14', 'E10'],
    y='hexatic_order_corrected',
    split=False,
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75,
    loc='outside',
    text_format='star'
)
annotator.apply_and_annotate()

plt.savefig(f"{figure_path}/Hexatic_order__E10_E14__tissue__stats.svg",format='svg')



In [ ]:
for stage in ['E8', 'E10', 'E12', 'E14']:
    fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
    fig.tight_layout()

    tmp_data = tissue_data.loc[tissue_data['stage'] == stage]
    merge_data = pd.DataFrame()
    for p in range(3, 10):
        __merge_data = pd.DataFrame()
        __merge_data['value'] = tmp_data[f'{p}-atic_order_corrected']
        __merge_data['p-atic_order'] = f'{p}-atic_order'
        if p == 6:
            __merge_data['hexatic_order'] = tmp_data['hexatic_order_corrected']
        else:
            __merge_data['hexatic_order'] = np.nan
        __merge_data['position'] = tmp_data['position']
        __merge_data['stage'] = tmp_data['stage']
        __merge_data['filename'] = tmp_data['filename']
        if not merge_data.empty:
            merge_data = merge_data.append(__merge_data)
        else:
            merge_data = __merge_data

    print(merge_data.loc[merge_data['p-atic_order']=='6-atic_order', ['filename', 'position', 'p-atic_order', 'value', 'hexatic_order']])
    print(merge_data.loc[merge_data['p-atic_order']=='6-atic_order', ['filename', 'position', 'p-atic_order', 'value', 'hexatic_order']].groupby(by='position').mean())
        

    points = sns.pointplot(
        data=merge_data,
        x='position',
        order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
        y='value',
        hue='p-atic_order',
        palette='hsv',
        color='red',
        join=False,
        errorbar='sd',
        capsize=.05,
        dodge=True,
        # s=2,
        errwidth=linewidth,
        markers='x',
    )
    plt.setp(points.collections, sizes=[markersize], linewidths=[2*linewidth])

    ax.set_ylim([0, 1.])
    ax.set_yticks([0., 0.5, 1.])
    ax.set_xlim([-0.5, 6.5])

    ax.set_ylabel(f'p-atic order $\psi_p^\star$')

    ax.set_xlabel('')
    ax.set_xticklabels(
        ['25I', '25S', '50I', '50S', '75I', '75S', '100'],
        fontsize=labelsize
    )

    # leg_handles = ax.get_legend_handles_labels()[0]
    # ax.legend(leg_handles, ['HC'], title='')

    plt.savefig(f"{figure_path}/p-atic_order__{stage}__tissue.svg",format='svg')

    

In [ ]:
for stage in ['E8', 'E10', 'E12', 'E14']:
    fig, ax = plt.subplots(1, 1, figsize=[2.2, 1.7])
    fig.tight_layout()

    points = sns.pointplot(
        data=tissue_data.loc[tissue_data['stage']==stage],
        x='position',
        order=['0', '25I', '25S', '50I', '50S', '75I', '75S', '100'],
        y='HC_density',
        color='red',
        join=False,
        estimator=np.mean,
        errorbar='sd',
        capsize=.05,
        dodge=True,
        scale=4,
        # s=10,
        errwidth=linewidth,
        markers='|',
    )
    plt.setp(points.collections, sizes=[2*markersize])
    points = sns.stripplot(
        data=tissue_data.loc[tissue_data['stage']==stage],
        x='position',
        order=['0', '25I', '25S', '50I', '50S', '75I', '75S', '100'],
        y='HC_density',
        color='red',
        dodge=True,
        alpha=0.8,
        size=20,
        # scale=5,
        # markers='.',
    )
    plt.setp(points.collections, sizes=[.25*markersize])

    ax.set_ylim([0, 0.4])
    ax.set_xlim([-0.5, 7.5])

    ax.set_ylabel('HC fraction')

    ax.set_xlabel('')
    ax.set_xticklabels(
        ['0', '25I', '25S', '50I', '50S', '75I', '75S', '100'], 
        fontsize=labelsize
    )

    # ax.legend()

    plt.savefig(f"{figure_path}/HC_density__{stage}.svg",format='svg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[2., 1.5])
fig.tight_layout()

points = sns.pointplot(
    data=tissue_data,
    x='position',
    order=['0', '25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='HC_density',
    hue='stage',
    palette=palette_stages,
    color='red',
    join=False,
    estimator=np.mean,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    scale=5,
    # s=10,markersize=20
    errwidth=linewidth/5,
    markers='|',
)
plt.setp(points.collections, sizes=[2*markersize])
points = sns.stripplot(
    data=tissue_data,
    x='position',
    order=['0', '25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='HC_density',
    hue='stage',
    palette=palette_stages,
    color='red',
    dodge=True,
    alpha=0.4,
    scale=5,
    # markers='.',
)
plt.setp(points.collections, sizes=[.25*markersize])

ax.set_ylim([0, 0.4])
ax.set_xlim([-0.5, 7.5])

ax.set_ylabel('HC fraction')

ax.set_xlabel('')
ax.set_xticklabels(
    ['0', '25I', '25S', '50I', '50S', '75I', '75S', '100'], 
    fontsize=labelsize
)

# ax.legend()

plt.savefig(f"{figure_path}/HC_density.svg",format='svg')

In [ ]:
pd.options.display.float_format = "{:.3f}".format
tissue_data.groupby(by=['stage', 'PD_position'])['HC_area__no_norm'].apply(lambda data: (int(data.mean()*1000)/1000, data.std(), int(data.min()*1000)/1000, int(data.max()*1000)/1000)).reset_index()

In [ ]:
pd.options.display.float_format = "{:.3f}".format
tissue_data.groupby(by=['stage', 'PD_position'])['HC_density'].apply(lambda data: (int(data.mean()*1000)/1000, data.std(), int(data.min()*1000)/1000, int(data.max()*1000)/1000)).reset_index()

In [ ]:
print(tissue_data[['stage', 'PD_position', 'HC_density']].sort_values(by=['stage', 'position', 'HC_density']).to_string())

In [ ]:
tissue_data.loc[tissue_data['PD_position']==25, 'HC_density'].mean()
tissue_data.loc[tissue_data['PD_position']==25, 'HC_density'].std()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[2.2, 1.6])
fig.tight_layout()

points = sns.pointplot(
    data=tissue_data.loc[
        (tissue_data['stage']=='E10') | (tissue_data['stage']=='E14')
    ],
    x='position',
    order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='HC_density',
    hue='stage',
    hue_order=['E10', 'E14'],
    palette=['gray', 'black'],
    join=False,
    # color='black',
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.set_ylim([0, 0.4])
ax.set_xlim([-0.5, 6.5])

ax.set_ylabel('HC number density')

ax.set_xlabel('')
ax.set_xticklabels(
    ['25I', '25S', '50I', '50S', '75I', '75S', '100'], 
    fontsize=labelsize
)

plt.savefig(f"{figure_path}/HC_density__E10_E14.svg",format='svg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/3, 5.9/3])

points = sns.pointplot(
    data=tissue_data.loc[tissue_data['stage'] == 'E14'],
    x='position',
    order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='number_ratio',
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.set_ylim([0.2, 0.8])
ax.hlines(y=[0.5], xmin=-0.5, xmax=6.5, colors=['black'], linewidth=linewidth)
ax.set_xlim([-0.5, 6.5])

ax.set_ylabel('HC / SC number ratio')

ax.set_xlabel('')
ax.set_xticklabels(['25I', '25S', '50I', '50S', '75I', '75S', '100'], fontsize=labelsize)

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['SC', 'HC'], title='')

plt.savefig(f"{figure_path}/HC_ratio__E14.svg",format='svg')

## Polarity

In [ ]:
fig, ax = plt.subplots(1, 1)


sns.pointplot(
    data=tissue_data,
    x='Stage',
    y='polarity_corrected__variance',
    hue='position', hue_order=['25I', '75I'],
    # palette='Oranges',
    join=False,
    ci='sd',
    # err_style='bars',
    capsize=.05,
    dodge=True,
)

# ax.set_xlim([0, 1])
ax.set_ylim([0, 1])

ax.set_ylabel('Circular variance of polarity (corr.)')



plt.legend(prop={'size': 'small'})

plt.savefig(f"{figure_path}/polarity_variance_vs_stage__explants.svg")

In [ ]:
results = pd.DataFrame()

_data = cells.loc[cells['is_HC']].copy()
_data = _data.loc[(_data['aligned_with_PD_axis']) | (_data['stage'] == 'E14')].copy()
_data = _data.loc[(_data['position'] == '25I') | (_data['position'] == '75I')].copy()

fig, axes = plt.subplots(
    1, 4,
    sharex=True, sharey=True,
    subplot_kw=dict(projection='polar'),
    figsize=[7., 3],
)
plt.subplots_adjust(hspace=0.)
(ax1, ax2, ax3, ax4) = axes
for i, (stage, ax) in enumerate(zip(['E8', 'E10', 'E12', 'E14'], axes)):
    plt.sca(ax)
    data = _data.loc[_data['stage'] == stage].copy()
    
    palette = ['tab:blue', 'tab:orange']
    __data = data
    tmp = data
    tmp['cilium_phi_corrected'] = data['cilium_phi_corrected'] + 2*math.pi
    __data = pd.concat([__data, tmp])
    tmp['cilium_phi_corrected'] = data['cilium_phi_corrected'] - 2*math.pi
    __data = __data.loc[__data['cilium_phi_corrected'] > - 1.2 * math.pi]
    __data = __data.loc[__data['cilium_phi_corrected'] <   1.2 * math.pi]
    __data = pd.concat([__data, tmp])
    sns.kdeplot(
        data=__data.reset_index(),
        x='cilium_phi_corrected',
        y='cilium_rho_corrected',
        hue='position',
        hue_order=['25I', '75I'],
        palette=palette,
        # bw_method=0.1,
        common_norm=False,
        fill=True,
        levels=3,
        thresh=0.2,
        alpha=0.4,
        zorder=10,
    )
    
    sns.scatterplot(
        data=data,
        x='cilium_phi_corrected',
        y='cilium_rho_corrected',
        hue='position',
        hue_order=['25I', '75I'],
        palette=palette,
        # style='file_id',
        alpha=0.75,
        s=3,
        edgecolors=None,
        ax=ax,
        zorder=11,
    )

    # sns.histplot(
    #     data=data,
    #     x='cilium_phi_corrected',
    #     weights='cilium_rho_corrected',
    #     hue='position',
    #     hue_order=['25I', '75I'],
    #     multiple='stack',
    #     bins=16,
    #     binrange=[-math.pi, math.pi],
    #     stat='probability',
    #     zorder=20,
    # )
    
    quiver_x = []
    quiver_y = []
    quiver_u = []
    quiver_v = []
    for pos in np.unique(data['position']):
        d = data.loc[data['position'] == pos]
        mean = circular_mean(d[['cilium_DX_corrected', 'cilium_DY_corrected']], eulerian=True)
        mean_length = circular_mean_length(d[['cilium_DX_corrected', 'cilium_DY_corrected']], eulerian=True, normalize=False)

        quiver_x.append(0)
        quiver_y.append(0)
        quiver_u.append(mean)
        quiver_v.append(mean_length)

    ax.quiver(
        quiver_x, quiver_y, quiver_u, quiver_v,
        facecolor=palette, edgecolor='black', linewidth=linewidth,
        scale=1, angles='xy', scale_units='xy',
        zorder=12
    )

    ax.invert_xaxis()
    ax.set_xlim(-math.pi, math.pi)
    ax.set_xticks([-math.pi, -math.pi / 2, 0, math.pi / 2])
    ax.set_xticklabels(['P', 'I', 'D', 'S'])
    ax.tick_params(axis='x', which='major', pad=-5)

    ax.set_ylim(0, 1.2)
    ax.set_yticks([0.5, 1.])

    ax.set_xlabel('')
    ax.set_ylabel('')

    ax.grid(zorder=-100)

    # if i == len(axes) - 1:
    #     mpl.pyplot.legend(bbox_to_anchor=(1.255,1.25))
    # else:
    #     ax.get_legend().remove()

plt.savefig(f"{figure_path}/polarity.svg")
        

# human readable information

grp = ['stage', 'position', 'file_id']
polarity_by_sample = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].mean()
polarity_by_sample['circular_mean'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean, eulerian=True)
polarity_by_sample['circular_mean_length'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean_length, eulerian=True, normalize=False)
polarity_by_sample['circular_stddev'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_stddev, eulerian=True)
polarity_by_sample['datapoints'] = _data.groupby(by=grp)['cilium_DX_corrected'].apply(lambda d: d.count())

polarity_by_sample['circular_mean__angles'] = (polarity_by_sample['circular_mean']/math.pi * 180 + 360) % 360
polarity_by_sample['circular_stddev__angles'] = (polarity_by_sample['circular_stddev']/math.pi * 180 + 360) % 360
print(polarity_by_sample.to_string())

grp = ['stage', 'position']
polarity_by_position = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].mean()
polarity_by_position['circular_mean'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean, eulerian=True)
polarity_by_position['circular_mean_length'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean_length, eulerian=True, normalize=False)
polarity_by_position['circular_stddev'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_stddev, eulerian=True)
polarity_by_position['datapoints'] = _data.groupby(by=grp)['cilium_DX_corrected'].apply(lambda d: d.count())

polarity_by_position['circular_mean__angles'] = (polarity_by_position['circular_mean']/math.pi * 180 + 360) % 360
polarity_by_position['circular_stddev__angles'] = (polarity_by_position['circular_stddev']/math.pi * 180 + 360) % 360
print(polarity_by_position.to_string())

grp = ['stage']
polarity_by_stage = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].mean()
polarity_by_stage['circular_mean'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean, eulerian=True)
polarity_by_stage['circular_mean_length'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean_length, eulerian=True, normalize=False)
polarity_by_stage['circular_stddev'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_stddev, eulerian=True)
polarity_by_stage['datapoints'] = _data.groupby(by=grp)['cilium_DX_corrected'].apply(lambda d: d.count())

polarity_by_stage['circular_mean__angles'] = (polarity_by_stage['circular_mean']/math.pi * 180 + 360) % 360
polarity_by_stage['circular_stddev__angles'] = (polarity_by_stage['circular_stddev']/math.pi * 180 + 360) % 360
print(polarity_by_stage.to_string())


In [ ]:
!pip install pycircstat
import pycircstat

!pip install hotelling
from hotelling.stats import hotelling_t2

In [ ]:
treatments = dict({
    'MLCK': [
        'MLCK',
        'MLCK_16hrs',
    ],
    'ROCK': [
        'ROCK',
        'ROCK_16hrs',
    ],
    'PTX': [
        'PTX'
    ],
    'Atoh1': [
        'Atoh1'
    ],
    'Vangl2': [
        'Vangl2'
    ]
})

figsizes=dict({
    'ROCK': 1.6,
    'Atoh1': 1.8,
})
# for Mode in ['aligned']:
for Mode in ['original', 'aligned', 'by_file', 'centred']:
    if Mode == 'by_file':
        mean_by_file=True
    else:
        mean_by_file=False

    for treatment, items in treatments.items():
        if treatment in figsizes:
            figsize=figsizes[treatment]
        else:
            figsize=1.4
        if len(items) > 1:
            fig, axes = plt.subplots(
                1, len(items),
                sharex=True, sharey=True,
                subplot_kw=dict(projection='polar'),
                figsize=[len(items)*figsize, figsize],
            )

            plt.subplots_adjust(hspace=0.)
        elif len(items) == 1:
            fig, ax = plt.subplots(
                1, 1,
                sharex=True, sharey=True,
                subplot_kw=dict(projection='polar'),
                figsize=[figsize, figsize],
            )
            axes = [ax]

        for i, ax in enumerate(axes):
            plt.sca(ax)
            data = cells_experiments.loc[cells_experiments['Experiment_Series']==items[i]].copy()
            data = data.sort_values(by='is_Control')

            N = data.groupby(by='is_Control')['cilium_phi_corrected'].count()
            markersize=3
            markersize_control=markersize
            markersize_inhibition=markersize
            if N.iloc[0] > 5 * N.iloc[1]:
                markersize_control *= 2
            elif 5 * N.iloc[0] > N.iloc[1]:
                markersize_inhibition *= 2

            # correct 
            if Mode == 'aligned':
                phi_column='cilium_phi_corrected__aligned'
                rho_column='cilium_rho_corrected'
                dx_column='cilium_DX_corrected__aligned'
                dy_column='cilium_DY_corrected__aligned'
            elif Mode == 'centred':
                phi_column='cilium_phi_corrected__centred'
                rho_column='cilium_rho_corrected'
                dx_column='cilium_DX_corrected__centred'
                dy_column='cilium_DY_corrected__centred'

            else:
                phi_column='cilium_phi_corrected'
                rho_column='cilium_rho_corrected'
                dx_column='cilium_DX_corrected'
                dy_column='cilium_DY_corrected'

            palette = ['tab:blue', 'tab:red'] # ['navy', 'orange']
            hue_order=[False, True]
            style=None
            if Mode == 'by_file':
                style='file_id'
            sns.scatterplot(
                data=data,
                x=phi_column,
                y=rho_column,
                hue='is_Control',
                hue_order=hue_order,
                size='is_Control',
                size_order=[True, False],
                sizes=[markersize_control, markersize_inhibition],
                style=style,
                palette=palette,
                alpha=0.75,
                # s=3,
                edgecolors=None,
                ax=ax,
                zorder=11,
            )
            __data = data.copy()

            # if Mode == 'aligned':
            if Mode == 'centred':
                print(
                    f"{items[i]} ({Mode}): p = %.2e"%pycircstat.watson_williams(
                        data.loc[data['is_Control']==True, phi_column].dropna(),
                        data.loc[data['is_Control']==False, phi_column].dropna()
                    )[0]
                )
                print(
                    f"{items[i]} ({Mode}): hotelling p = %.2e"%hotelling_t2(
                        data.loc[data['is_Control']==True, [dx_column, dy_column]].dropna(),
                        data.loc[data['is_Control']==False, [dx_column, dy_column]].dropna()
                    )[2]
                )
                print(
                    f"Uniformity Control: p = %.2e"%pycircstat.rayleigh(
                        data.loc[data['is_Control']==True, phi_column].dropna(),
                    )[0]
                )
                print(
                    f"Uniformity treatment: p = %.2e"%pycircstat.rayleigh(
                        data.loc[data['is_Control']==False, phi_column].dropna()
                    )[0]
                )
                print()

            tmp = data.loc[data[phi_column] < -1.2 * math.pi].copy()
            tmp[phi_column] = tmp[phi_column] + 2*math.pi
            __data = pd.concat([__data, tmp])

            tmp = data.loc[data[phi_column] > 1.2 * math.pi].copy()
            tmp[phi_column] = tmp[phi_column] - 2*math.pi
            __data = pd.concat([__data, tmp])

            sns.kdeplot(
                data=__data.reset_index(),
                x=phi_column,
                y=rho_column,
                hue='is_Control',
                hue_order=hue_order,
                palette=palette,
                fill=True,
                common_norm=False,
                levels=3,
                thresh=.2,
                alpha=0.4,
                zorder=10,
            )
            
            quiver_x = []
            quiver_y = []
            quiver_u = []
            quiver_v = []
            color = []
            for j, is_control in enumerate(hue_order):
                d = data.loc[data['is_Control'] == is_control]

                # if treatment == 'Vangl2':
                if mean_by_file:
                    mean_phi = d.groupby(by='file_id')[[dx_column, dy_column]].apply(circular_mean, eulerian=True)
                    mean_rho = d.groupby(by='file_id')[[dx_column, dy_column]].apply(circular_mean_length, eulerian=True, normalize=False)

                else:
                    mean_phi = [circular_mean(d[[dx_column, dy_column]], eulerian=True)]
                    mean_rho = [circular_mean_length(d[[dx_column, dy_column]], eulerian=True, normalize=False)]
                
                for phi, rho in zip(mean_phi, mean_rho):
                    quiver_x.append(0)
                    quiver_y.append(0)
                    quiver_u.append(phi)
                    quiver_v.append(rho)
                    color.append(palette[j])

            ax.quiver(
                quiver_x, quiver_y, quiver_u, quiver_v,
                facecolor=color, edgecolor='black', linewidth=linewidth,
                scale=1, angles='xy', scale_units='xy',
                zorder=12
            )

            ax.set_xlim(-math.pi, math.pi)
            ax.set_xticks([-math.pi, -math.pi / 2, 0, math.pi / 2])
            ax.set_xticklabels(['P', 'I', 'D', 'S'])
            ax.tick_params(axis='x', which='major', pad=-5)

            ax.set_ylim(0, 1.2)
            ax.set_yticks([0.5, 1.])

            ax.set_xlabel('')
            ax.set_ylabel('')

            if i == len(axes) - 1:
                leg_handles = ax.get_legend_handles_labels()[0]
                mpl.pyplot.legend(
                    leg_handles[::-1],
                    ['Control', f'{treatment}'],
                    title='',
                    fontsize=ticksize,
                    bbox_to_anchor=(1.05,1.05),
                    markerscale=1
                )
            else:
                ax.get_legend().remove()

        plt.savefig(f"{figure_path}/polarity__{treatment}__{Mode}.svg")

## Atoh1 density

In [ ]:

fix, ax = plt.subplots(1, 1, figsize=[1.2, 1.7])

sns.violinplot(
    data=tissue_data__experiments,
    x='Treatment',
    order=['Atoh1control', 'Atoh1KO'],
    palette=['tab:red', 'tab:blue'],
    y='HC_density',
    inner='point',
    linewidth=linewidth
)

box_pairs=[
    ('Atoh1control', 'Atoh1KO'),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=tissue_data__experiments,
    x='Treatment',
    order=['Atoh1control', 'Atoh1KO'],
    y='HC_density',
    split=False,
)
text_format='star'
annotator.configure(
    test='Mann-Whitney',
    line_width=0.25,
    loc='outside',
    text_format=text_format
)
annotator.apply_and_annotate()


ax.set_xlabel('')
ax.set_xticklabels(['Control', 'Atoh1-KO'], rotation=30, ha='right', rotation_mode='anchor')

ax.set_ylabel('HC density')
ax.set_ylim(0, 0.4)
ax.set_yticks([0, 0.2, 0.4])

plt.savefig(f"{figure_path}/HC_density_Atoh1.svg")

# Supplementary information

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

points = sns.pointplot(
    data=tissue_data,
    x='stage',
    y='elongation_HC',
    hue='position', hue_order=['25I', '75I'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.'
)
points = sns.scatterplot(
    data=tissue_data,
    x='stage',
    y='elongation_HC',
    hue='position', hue_order=['25I', '75I'],
    markers='.'
)
# points = sns.pointplot(
#     data=tissue_data,
#     x='stage',
#     y='elongation_HC',
#     hue='position', hue_order=['25I', '75I'],
#     join=False,
#     errorbar='sd',
#     capsize=.05,
#     dodge=True,
#     errwidth=linewidth,
#     markers='x'
# )
# points = sns.pointplot(
#     data=tissue_data,
#     x='stage',
#     y='elongation_SC',
#     hue='position', hue_order=['25I', '75I'],
#     join=False,
#     errorbar='sd',
#     capsize=.05,
#     dodge=True,
#     errwidth=linewidth,
#     markers='o'
# )
# points = sns.scatterplot(
#     data=tissue_data,
#     x='stage',
#     y='elongation_cells',
#     hue='position', hue_order=['25I', '75I'],
#     markers='x',
#     # palette=[''],
#     join=False,
#     errorbar='sd',
#     capsize=.05,
#     dodge=True,
#     errwidth=linewidth,
#     markers='.',
# )
plt.setp(points.collections, sizes=[markersize])

print(HC.loc[HC['position'] == '25I'].groupby(by='stage')['normalized_area_cells'].mean())

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['25I', '75I'], title='')


ax.set_ylim([0, None])
ax.set_xlim([-0.5, 3.5])

# ax.set_ylabel('Normalised HC area')
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)


plt.savefig(f"{figure_path}/HC_elongation.svg",format='svg')

In [ ]:
print("Available datasets: ")
for set in cells.columns:
  print("  {}".format(set))

In [ ]:
print("Available datasets in bonds: ")
for set in bonds.columns:
  print("  {}".format(set))